In [ ]:
# @title Default title text
# Google Colab Flask Video Processor Setup
# Run each cell in sequence

# ===== CELL 1: Install Dependencies =====
# ===== CELL 1: Installation with specific versions =====
!pip install flask flask-cors pyngrok
!pip install transformers openpyxl pandas
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install banglanlptoolkit
!pip install moviepy
!pip install SpeechRecognition==3.10.0
!pip install pydub==0.25.1

!apt-get update
!apt-get install -y ffmpeg

# ===== CELL 2: Setup Ngrok (for public URL) =====
from pyngrok import ngrok
import getpass

# Set your ngrok auth token
ngrok_token = getpass.getpass("Enter your ngrok auth token: ")
ngrok.set_auth_token(ngrok_token)

# ===== CELL 3: Fixed Flask Application =====
from flask import Flask, request, jsonify, send_file, render_template_string
from flask_cors import CORS
import os
import io
import json
import tempfile
import threading
import time
from datetime import datetime
import traceback
import zipfile
import re
import shutil

# Import libraries
import speech_recognition as sr
from tqdm.auto import tqdm
from pydub import AudioSegment
from pydub.utils import make_chunks
from moviepy.editor import VideoFileClip
import pandas as pd
import torch
import re
import gc
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from moviepy.editor import VideoFileClip
import math
from collections import Counter

# Try to import BNLP toolkit (optional)
try:
    from bnlp import BasicTokenizer
    try:
        from bnlp.stemmer import BanglaStemmer
    except ImportError:
        BanglaStemmer = None
    TOKENIZER = BasicTokenizer()
    STEMMER = BanglaStemmer() if BanglaStemmer else None
except ImportError:
    TOKENIZER = None
    STEMMER = None




# Enable tqdm for pandas
tqdm.pandas()

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

def split_into_sentences(text):
    """Split Bangla text into sentences"""
    if not text or not isinstance(text, str):
        return []

    # Remove extra whitespace
    text = ' '.join(text.split())

    # Split by Bangla sentence endings
    sentences = re.split(r'[।?!]+', text)

    # Clean and filter sentences
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence and len(sentence) > 3:  # Minimum sentence length
            cleaned_sentences.append(sentence)

    return cleaned_sentences

class EfficientCombinedTFIDF:
    def __init__(self):
        self.document_unigram_counts = []  # List of Counter objects for unigrams
        self.document_bigram_counts = []   # List of Counter objects for bigrams
        self.document_names = []
        self.global_unigram_vocab = set()
        self.global_bigram_vocab = set()
        self.unigram_document_frequencies = {}
        self.bigram_document_frequencies = {}
        self.total_documents = 0
        self.original_forms = {}  # Mapping of normalized -> original forms

    def add_document(self, doc_name, normalized_unigrams_list, normalized_bigrams_list, original_forms_mapping):
        """
        Add a document's unigrams and bigrams (already processed and normalized)
        """
        # Count unigrams and bigrams in this document
        unigram_counts = Counter(normalized_unigrams_list)
        bigram_counts = Counter(normalized_bigrams_list)

        self.document_unigram_counts.append(unigram_counts)
        self.document_bigram_counts.append(bigram_counts)
        self.document_names.append(doc_name)

        # Update global vocabularies
        unique_unigrams_in_doc = set(unigram_counts.keys())
        unique_bigrams_in_doc = set(bigram_counts.keys())

        self.global_unigram_vocab.update(unique_unigrams_in_doc)
        self.global_bigram_vocab.update(unique_bigrams_in_doc)

        # Update document frequencies
        for unigram in unique_unigrams_in_doc:
            self.unigram_document_frequencies[unigram] = self.unigram_document_frequencies.get(unigram, 0) + 1

        for bigram in unique_bigrams_in_doc:
            self.bigram_document_frequencies[bigram] = self.bigram_document_frequencies.get(bigram, 0) + 1

        self.total_documents += 1

        # Store original forms mapping
        self.original_forms.update(original_forms_mapping)

        print(f"Added document: {doc_name} with {len(normalized_unigrams_list)} unigrams, {len(normalized_bigrams_list)} bigrams")

    def calculate_pmi(self, bigram, doc_index):
        """
        Calculate Pointwise Mutual Information for a bigram
        """
        if ' ' not in bigram:
            return 0  # Not a bigram

        word1, word2 = bigram.split(' ', 1)

        # Get counts from current document
        unigram_counts = self.document_unigram_counts[doc_index]
        bigram_counts = self.document_bigram_counts[doc_index]

        # Calculate total tokens in document
        total_unigrams = sum(unigram_counts.values())

        # Get frequencies
        bigram_freq = bigram_counts.get(bigram, 0)
        word1_freq = unigram_counts.get(word1, 0)
        word2_freq = unigram_counts.get(word2, 0)

        if bigram_freq == 0 or word1_freq == 0 or word2_freq == 0:
            return 0

        # Calculate probabilities
        p_bigram = bigram_freq / total_unigrams
        p_word1 = word1_freq / total_unigrams
        p_word2 = word2_freq / total_unigrams

        # Calculate PMI
        if p_word1 * p_word2 > 0:
            pmi = math.log(p_bigram / (p_word1 * p_word2))
            return pmi
        else:
            return 0

    def decide_unigram_vs_bigram(self, bigram, doc_index, min_pmi=1.0, min_freq=3):
        """
        Decide whether to keep unigrams or bigram based on PMI and frequency
        Returns: ('keep_bigram', 'keep_unigrams', 'keep_both') and decision reason
        """
        if ' ' not in bigram:
            return 'keep_unigrams', 'Not a valid bigram'

        words = bigram.split(' ', 1)
        if len(words) != 2:
            return 'keep_unigrams', 'Invalid bigram format'

        word1, word2 = words

        # Get counts
        bigram_counts = self.document_bigram_counts[doc_index]
        unigram_counts = self.document_unigram_counts[doc_index]

        bigram_freq = bigram_counts.get(bigram, 0)
        word1_freq = unigram_counts.get(word1, 0)
        word2_freq = unigram_counts.get(word2, 0)

        # Check minimum frequency threshold
        if bigram_freq < min_freq:
            return 'keep_unigrams', f'Bigram too rare (freq={bigram_freq})'

        # Calculate PMI
        pmi = self.calculate_pmi(bigram, doc_index)

        # Decision logic
        if pmi > min_pmi:
            # Strong association - keep bigram, remove constituent unigrams
            return 'keep_bigram', f'Strong association (PMI={pmi:.2f})'
        elif pmi > 0.5:
            # Moderate association - keep both
            return 'keep_both', f'Moderate association (PMI={pmi:.2f})'
        else:
            # Weak association - keep individual unigrams
            return 'keep_unigrams', f'Weak association (PMI={pmi:.2f})'

    def calculate_tfidf_with_decisions(self, doc_index, top_k=None, percentage_based=True,
                                     base_percentage=2.0, min_keywords=5, max_keywords=50):
        """
        Calculate TF-IDF scores with unigram vs bigram decisions
        """
        if doc_index >= len(self.document_unigram_counts):
            return None

        unigram_counts = self.document_unigram_counts[doc_index]
        bigram_counts = self.document_bigram_counts[doc_index]

        total_unigrams = sum(unigram_counts.values())

        # Calculate dynamic top_k if not provided
        if top_k is None and percentage_based:
            calculated_k = int(total_unigrams * (base_percentage / 100))
            top_k = max(min_keywords, min(calculated_k, max_keywords))
            print(f"Document length: {total_unigrams} tokens, Dynamic keywords: {top_k}")
        elif top_k is None:
            top_k = 20

        # Process all terms and make decisions
        final_keywords = []
        eliminated_unigrams = set()  # Track which unigrams to skip
        decision_log = []

        # First pass: Process all bigrams and make decisions
        for bigram, count in bigram_counts.items():
            decision, reason = self.decide_unigram_vs_bigram(bigram, doc_index)

            if decision == 'keep_bigram':
                # Calculate TF-IDF for bigram
                tf = count / total_unigrams
                df = self.bigram_document_frequencies[bigram]
                idf = math.log(self.total_documents / df)
                tfidf_score = tf * idf

                # Get original forms
                bigram_words = bigram.split()
                original_parts = []
                has_mappings = False

                for word in bigram_words:
                    if word in self.original_forms and len(self.original_forms[word]) > 1:
                        original_parts.append(f"{word}←{sorted(list(self.original_forms[word]))}")
                        has_mappings = True
                    else:
                        original_parts.append(word)

                original_forms = " | ".join(original_parts) if has_mappings else bigram

                final_keywords.append({
                    'Keyword': bigram,
                    'TF-IDF_Score': round(tfidf_score, 4),
                    'TF': round(tf, 4),
                    'IDF': round(idf, 4),
                    'Count_in_Doc': count,
                    'IDF_Count': df,
                    'Term_Type': 'Bigram',
                    'Decision': decision,
                    'Decision_Reason': reason,
                    'PMI': round(self.calculate_pmi(bigram, doc_index), 3),
                    'Original_Forms': original_forms
                })

                # Mark constituent unigrams for elimination
                word1, word2 = bigram.split(' ', 1)
                eliminated_unigrams.add(word1)
                eliminated_unigrams.add(word2)

            elif decision == 'keep_both':
                # Calculate TF-IDF for bigram
                tf = count / total_unigrams
                df = self.bigram_document_frequencies[bigram]
                idf = math.log(self.total_documents / df)
                tfidf_score = tf * idf

                # Get original forms
                bigram_words = bigram.split()
                original_parts = []
                has_mappings = False

                for word in bigram_words:
                    if word in self.original_forms and len(self.original_forms[word]) > 1:
                        original_parts.append(f"{word}←{sorted(list(self.original_forms[word]))}")
                        has_mappings = True
                    else:
                        original_parts.append(word)

                original_forms = " | ".join(original_parts) if has_mappings else bigram

                final_keywords.append({
                    'Keyword': bigram,
                    'TF-IDF_Score': round(tfidf_score, 4),
                    'TF': round(tf, 4),
                    'IDF': round(idf, 4),
                    'Count_in_Doc': count,
                    'IDF_Count': df,
                    'Term_Type': 'Bigram',
                    'Decision': decision,
                    'Decision_Reason': reason,
                    'PMI': round(self.calculate_pmi(bigram, doc_index), 3),
                    'Original_Forms': original_forms
                })

                # Don't eliminate unigrams in this case

            decision_log.append((bigram, decision, reason))

        # Second pass: Process remaining unigrams
        for unigram, count in unigram_counts.items():
            if unigram not in eliminated_unigrams:
                # Calculate TF-IDF for unigram
                tf = count / total_unigrams
                df = self.unigram_document_frequencies[unigram]
                idf = math.log(self.total_documents / df)
                tfidf_score = tf * idf

                # Get original forms
                original_forms = unigram
                if unigram in self.original_forms and len(self.original_forms[unigram]) > 1:
                    original_forms = f"{unigram}←{sorted(list(self.original_forms[unigram]))}"

                final_keywords.append({
                    'Keyword': unigram,
                    'TF-IDF_Score': round(tfidf_score, 4),
                    'TF': round(tf, 4),
                    'IDF': round(idf, 4),
                    'Count_in_Doc': count,
                    'IDF_Count': df,
                    'Term_Type': 'Unigram',
                    'Decision': 'keep_unigrams',
                    'Decision_Reason': 'No competing bigram or bigram rejected',
                    'PMI': 0.0,
                    'Original_Forms': original_forms
                })

        # Sort by TF-IDF score
        final_keywords.sort(key=lambda x: x['TF-IDF_Score'], reverse=True)

        # Calculate relative importance
        if final_keywords:
            max_tfidf = final_keywords[0]['TF-IDF_Score']
            for item in final_keywords:
                item['Relative_Importance'] = round(item['TF-IDF_Score'] / max_tfidf, 4) if max_tfidf > 0 else 0

        # Create results DataFrame
        results_df = pd.DataFrame(final_keywords[:top_k])
        results_df.attrs['document_name'] = self.document_names[doc_index]
        results_df.attrs['document_index'] = doc_index
        results_df.attrs['total_keywords_available'] = len(final_keywords)
        results_df.attrs['document_length'] = total_unigrams
        results_df.attrs['eliminated_unigrams'] = len(eliminated_unigrams)
        results_df.attrs['decision_log'] = decision_log

        return results_df

    def analyze_all_documents(self, top_k=None, save_to_excel=True, output_path=None):
        """
        Analyze keywords for all documents with unigram vs bigram decisions
        """
        print(f"Analyzing combined unigram-bigram keywords for all {self.total_documents} documents...")

        all_analyses = {}
        excel_data = []
        decision_stats = {'keep_bigram': 0, 'keep_unigrams': 0, 'keep_both': 0}

        for doc_idx in range(self.total_documents):
            keywords_df = self.calculate_tfidf_with_decisions(doc_idx, top_k=top_k)

            if keywords_df is not None and not keywords_df.empty:
                doc_name = keywords_df.attrs.get('document_name', f'Document_{doc_idx}')
                all_analyses[doc_name] = keywords_df

                # Collect decision statistics
                decision_log = keywords_df.attrs.get('decision_log', [])
                for _, decision, _ in decision_log:
                    decision_stats[decision] = decision_stats.get(decision, 0) + 1

                # Prepare data for Excel
                for idx, row in keywords_df.iterrows():
                    excel_row = {
                        'Document_Name': doc_name,
                        'Document_Index': doc_idx,
                        'Rank': idx + 1,
                        **row.to_dict()
                    }
                    excel_data.append(excel_row)

            if (doc_idx + 1) % 25 == 0:
                print(f"Processed {doc_idx + 1}/{self.total_documents} documents...")

        print("✓ Combined unigram-bigram analysis complete!")

        # Print decision statistics
        print(f"\nDecision Statistics:")
        print(f"  - Keep bigram (eliminate unigrams): {decision_stats.get('keep_bigram', 0)}")
        print(f"  - Keep unigrams (eliminate bigram): {decision_stats.get('keep_unigrams', 0)}")
        print(f"  - Keep both: {decision_stats.get('keep_both', 0)}")

        if save_to_excel and excel_data:
            if output_path is None:
                output_path = '/content/drive/MyDrive/tfidf/all_document_combined_keywords.xlsx'

            try:
                # Create Excel file with multiple sheets
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    # Summary sheet
                    summary_df = pd.DataFrame(excel_data)
                    summary_df.to_excel(writer, sheet_name='All_Documents_Combined', index=False)

                    # Individual document sheets (first 10 documents)
                    for i, (doc_name, keywords_df) in enumerate(list(all_analyses.items())[:10]):
                        sheet_name = f'Doc_{i+1}'[:31]  # Excel sheet name limit
                        keywords_df.to_excel(writer, sheet_name=sheet_name, index=False)

                    # Decision summary sheet
                    decision_df = pd.DataFrame([
                        {'Decision_Type': k, 'Count': v} for k, v in decision_stats.items()
                    ])
                    decision_df.to_excel(writer, sheet_name='Decision_Summary', index=False)

                print(f"Results saved to: {output_path}")

            except Exception as e:
                print(f"Error saving to Excel: {e}")

        return all_analyses

    def get_vocabulary_stats(self):
        """Get statistics about the vocabulary"""
        total_unigrams = sum(sum(doc_counts.values()) for doc_counts in self.document_unigram_counts)
        total_bigrams = sum(sum(doc_counts.values()) for doc_counts in self.document_bigram_counts)

        avg_unigrams = total_unigrams / self.total_documents if self.total_documents > 0 else 0
        avg_bigrams = total_bigrams / self.total_documents if self.total_documents > 0 else 0

        return {
            'total_documents': self.total_documents,
            'unique_unigrams': len(self.global_unigram_vocab),
            'unique_bigrams': len(self.global_bigram_vocab),
            'total_unigrams': total_unigrams,
            'total_bigrams': total_bigrams,
            'avg_unigrams_per_doc': round(avg_unigrams, 1),
            'avg_bigrams_per_doc': round(avg_bigrams, 1),
            'terms_with_mappings': len(self.original_forms)
        }

class EnhancedBanglaCombinedKeywordExtractor:
    def __init__(self, stopwords_file_path='/content/drive/MyDrive/tfidf/stop.xlsx'):
        """
        Initialize the Enhanced Bangla Combined (Unigram + Bigram) Keyword Extractor
        """
        try:
            if BasicTokenizer is None:
                raise ImportError("BasicTokenizer not available")

            self.tokenizer = BasicTokenizer()

            if BanglaStemmer is not None:
                self.stemmer = BanglaStemmer()
                self.use_stemmer = True
                print("Tokenizer and Stemmer initialized successfully")
            else:
                self.stemmer = None
                self.use_stemmer = False
                print("Tokenizer initialized successfully (Stemmer not available - will use suffix removal)")

        except Exception as e:
            print(f"Error initializing BNLP components: {e}")
            self.tokenizer = None
            self.stemmer = None
            self.use_stemmer = False
            print("Using fallback tokenization (simple split)")

        # Common Bangla suffixes for normalization (sorted by length for proper matching)
        self.bangla_suffixes = [
            'গুলোকে','গুলিকে','গুলাকে','গুলো', 'গুলি', 'গুলা', 'েরা', 'েদের', 'দের', 'রা',
            'এর', 'ের', 'র', 'কে', 'তে', 'য়ে','বান', 'মান',
            'খানা', 'খানি', 'টুকু', 'টিকে', 'টাকে', 'টি', 'টা',
            'ওয়ালা', 'ওয়ালি', 'ে'
        ]
        self.bangla_suffixes.sort(key=len, reverse=True)

        # Initialize stopwords
        self.stopwords = set()
        if stopwords_file_path:
            self._load_custom_stopwords(stopwords_file_path)

        # Initialize variables
        self.document_paths = []
        self.raw_documents = []
        self.vocabulary_mapping = {}

        # Initialize the efficient TF-IDF calculator
        self.tfidf_calculator = EfficientCombinedTFIDF()

    def _load_custom_stopwords(self, file_path):
        """Load custom stopwords from Excel file"""
        try:
            df = pd.read_excel(file_path, header=None)
            custom_stops = df.iloc[:, 0].dropna().astype(str).str.strip().tolist()
            custom_stops = [stop for stop in custom_stops if stop]

            # Tokenize stopwords only if tokenizer is available
            if self.tokenizer and BasicTokenizer is not None:
                tokenized_stopwords = set()
                for stop in custom_stops:
                    tokens = self.tokenizer.tokenize(stop)
                    tokenized_stopwords.update(tokens)
                self.stopwords.update(tokenized_stopwords)
                print(f"Successfully loaded and tokenized {len(custom_stops)} stopwords")
            else:
                # If no tokenizer, keep stopwords as full words
                self.stopwords.update(custom_stops)
                print(f"Successfully loaded {len(custom_stops)} stopwords (no tokenization - tokenizer not available)")

        except FileNotFoundError:
            print(f"Error: Stopwords file not found at {file_path}")
        except Exception as e:
            print(f"Error loading custom stopwords: {e}")

    def smart_suffix_removal(self, word, corpus_vocabulary=None):
        """
        Smart suffix removal that checks if the root exists in corpus
        """
        if len(word) < 3:
            return word, False

        # Try stemmer first if available
        if self.use_stemmer and self.stemmer is not None:
            try:
                stem = self.stemmer.stem(word)
                if stem and len(stem) > 1:
                    if corpus_vocabulary is None or stem in corpus_vocabulary:
                        return stem, True
                    else:
                        return word, False  # Keep original if stem not prominent
            except Exception:
                pass

        # Fallback to suffix removal
        for suffix in self.bangla_suffixes:
            if word.endswith(suffix) and len(word) > len(suffix) + 1:
                root = word[:-len(suffix)]
                if len(root) > 1:
                    # Check if root exists in corpus vocabulary
                    if corpus_vocabulary is None or root in corpus_vocabulary:
                        return root, True
                    else:
                        return word, False  # Keep original

        return word, False

    def create_unigrams_and_bigrams_from_raw(self, text):
        """
        Create both unigrams and bigrams from raw text within sentences only
        """
        if not text or not isinstance(text, str):
            return [], []

        # Split by sentence delimiters
        sentences = re.split(r'[।!?]+', text)
        all_unigrams = []
        all_bigrams = []

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            # Clean sentence but keep basic structure
            sentence = re.sub(r'[^\u0980-\u09FF\s]', ' ', sentence)
            sentence = re.sub(r'\s+', ' ', sentence).strip()

            if not sentence:
                continue

            # Tokenize the sentence
            try:
                if self.tokenizer is not None:
                    tokens = self.tokenizer.tokenize(sentence)
                else:
                    tokens = sentence.split()
            except Exception:
                tokens = sentence.split()

            # Filter tokens (basic filtering, no stopword removal yet)
            filtered_tokens = []
            for token in tokens:
                if token and len(token) >= 2:
                    filtered_tokens.append(token)

            # Create unigrams
            all_unigrams.extend(filtered_tokens)

            # Create bigrams within this sentence
            for i in range(len(filtered_tokens) - 1):
                bigram = (filtered_tokens[i], filtered_tokens[i + 1])
                all_bigrams.append(bigram)

        return all_unigrams, all_bigrams

    def build_corpus_vocabulary(self, min_frequency=3):
        """
        Build a vocabulary of prominent terms from all documents
        """
        all_tokens = []

        # First pass: collect all tokens with basic processing
        for doc_path in self.document_paths:
            try:
                with open(doc_path, 'r', encoding='utf-8') as file:
                    content = file.read()

                # Basic text cleaning
                content = re.sub(r'[^\u0980-\u09FF\s।!?]', ' ', content)
                content = re.sub(r'\s+', ' ', content).strip()

                if self.tokenizer is not None:
                    tokens = self.tokenizer.tokenize(content)
                else:
                    tokens = content.split()

                # Basic filtering
                filtered_tokens = []
                for token in tokens:
                    if len(token) >= 2 and not re.match(r'^[।!?]+$', token):
                        filtered_tokens.append(token)

                all_tokens.extend(filtered_tokens)

            except Exception as e:
                print(f"Error processing {doc_path}: {e}")

        # Count frequencies and find prominent terms
        token_counts = Counter(all_tokens)
        prominent_terms = {token for token, count in token_counts.items() if count >= min_frequency}

        print(f"Built corpus vocabulary: {len(prominent_terms)} prominent terms")
        return prominent_terms

    def preprocess_text_combined(self, text, corpus_vocabulary):
        """
        Preprocess text to create filtered and normalized unigrams and bigrams
        """
        if not text or not isinstance(text, str):
            return [], [], {}

        # Step 1: Create unigrams and bigrams from raw text
        raw_unigrams, raw_bigrams = self.create_unigrams_and_bigrams_from_raw(text)

        if not raw_unigrams and not raw_bigrams:
            return [], [], {}

        # Step 2: Process unigrams
        processed_unigrams = []
        mapping_info = {}

        for word in raw_unigrams:
            # Normalize
            norm_word, was_merged = self.smart_suffix_removal(word, corpus_vocabulary)

            # Check stopword after normalization
            if norm_word not in self.stopwords:
                processed_unigrams.append(norm_word)

                # Update mapping info
                if was_merged and norm_word != word:
                    if norm_word not in mapping_info:
                        mapping_info[norm_word] = set()
                    mapping_info[norm_word].add(word)

        # Step 3: Process bigrams
        processed_bigrams = []

        for word1, word2 in raw_bigrams:
            # Normalize both words
            norm_word1, was_merged1 = self.smart_suffix_removal(word1, corpus_vocabulary)
            norm_word2, was_merged2 = self.smart_suffix_removal(word2, corpus_vocabulary)

            # Check stopwords after normalization
            if norm_word1 not in self.stopwords and norm_word2 not in self.stopwords:
                bigram_string = f"{norm_word1} {norm_word2}"
                processed_bigrams.append(bigram_string)

                # Update mapping info
                if was_merged1 and norm_word1 != word1:
                    if norm_word1 not in mapping_info:
                        mapping_info[norm_word1] = set()
                    mapping_info[norm_word1].add(word1)

                if was_merged2 and norm_word2 != word2:
                    if norm_word2 not in mapping_info:
                        mapping_info[norm_word2] = set()
                    mapping_info[norm_word2].add(word2)

        return processed_unigrams, processed_bigrams, mapping_info

    def load_documents_from_folder(self, folder_path):
        """
        Load and preprocess all .txt files with combined unigram and bigram processing
        """
        self.document_paths = []
        self.raw_documents = []
        self.vocabulary_mapping = {}

        if not os.path.exists(folder_path):
            print(f"Error: Folder {folder_path} does not exist")
            return

        txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

        if not txt_files:
            print(f"No .txt files found in {folder_path}")
            return

        print(f"Found {len(txt_files)} .txt files.")

        # First, build corpus vocabulary
        print("Building corpus vocabulary...")
        self.document_paths = [os.path.join(folder_path, f) for f in txt_files]
        corpus_vocabulary = self.build_corpus_vocabulary(min_frequency=2)

        # Now process documents with combined processing
        print("Processing documents with combined unigram-bigram extraction...")
        processed_count = 0

        for i, filename in enumerate(txt_files):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()

                self.raw_documents.append(content)

                # Combined preprocessing
                processed_unigrams, processed_bigrams, doc_mappings = self.preprocess_text_combined(content, corpus_vocabulary)

                if processed_unigrams or processed_bigrams:
                    # Add to efficient TF-IDF calculator
                    doc_name = filename
                    self.tfidf_calculator.add_document(doc_name, processed_unigrams, processed_bigrams, doc_mappings)

                    # Update global vocabulary mapping
                    for normalized, originals in doc_mappings.items():
                        if normalized not in self.vocabulary_mapping:
                            self.vocabulary_mapping[normalized] = set()
                        self.vocabulary_mapping[normalized].update(originals)

                    processed_count += 1
                    if processed_count % 20 == 0:
                        print(f"Processed {processed_count}/{len(txt_files)} documents...")
                else:
                    print(f"⚠ Skipped (empty after processing): {filename}")

            except Exception as e:
                print(f"✗ Error processing {filename}: {e}")

        print(f"\nSuccessfully processed {processed_count} documents")
        print(f"Vocabulary mappings created: {len(self.vocabulary_mapping)} terms")

        # Show some mapping examples
        if self.vocabulary_mapping:
            print("\nSample suffix mappings:")
            for normalized, originals in list(self.vocabulary_mapping.items())[:10]:
                if len(originals) > 1:
                    print(f"  {normalized} <- {list(originals)}")

    def get_keywords_for_document(self, doc_index, top_k=None):
        """
        Get keywords for a specific document using the combined calculator with decisions
        """
        return self.tfidf_calculator.calculate_tfidf_with_decisions(doc_index, top_k=top_k)

    def analyze_all_documents(self, top_k=None, save_to_excel=True, output_path=None):
        """
        Analyze keywords for all documents using the combined calculator with decisions
        """
        return self.tfidf_calculator.analyze_all_documents(top_k=top_k, save_to_excel=save_to_excel, output_path=output_path)

    def get_statistics(self):
        """Get overall statistics"""
        stats = self.tfidf_calculator.get_vocabulary_stats()
        stats['terms_with_suffix_mappings'] = len(self.vocabulary_mapping)
        return stats

    def get_decision_analysis(self, doc_index):
        """
        Get detailed decision analysis for a specific document
        """
        keywords_df = self.get_keywords_for_document(doc_index)
        if keywords_df is None:
            return None

        decision_log = keywords_df.attrs.get('decision_log', [])

        analysis = {
            'document_name': keywords_df.attrs.get('document_name'),
            'eliminated_unigrams': keywords_df.attrs.get('eliminated_unigrams', 0),
            'total_keywords': keywords_df.attrs.get('total_keywords_available', 0),
            'decisions': {}
        }

        for bigram, decision, reason in decision_log:
            analysis['decisions'][bigram] = {
                'decision': decision,
                'reason': reason,
                'pmi': round(self.tfidf_calculator.calculate_pmi(bigram, doc_index), 3)
            }

        return analysis

# Enhanced main function for combined analysis
def enhanced_combined_main():
    """Enhanced main function for combined unigram-bigram analysis with PMI decisions"""

    # Initialize the enhanced extractor
    print("Initializing Enhanced Bangla Combined Keyword Extractor...")
    extractor = EnhancedBanglaCombinedKeywordExtractor(
        stopwords_file_path='/content/drive/MyDrive/tfidf/temp.xlsx'
    )

    # Load documents
    folder_path = '/content/drive/MyDrive/tfidf/Concatenated_Summary'
    print(f"\nLoading documents from: {folder_path}")
    extractor.load_documents_from_folder(folder_path)

    if extractor.tfidf_calculator.total_documents == 0:
        print("No documents were processed. Exiting.")
        return

    # Analyze all documents and save to Excel
    print(f"\n{'='*60}")
    print("ANALYZING ALL DOCUMENTS - COMBINED UNIGRAM-BIGRAM WITH PMI DECISIONS")
    print(f"{'='*60}")

    all_results = extractor.analyze_all_documents(
        save_to_excel=True,
        output_path='/content/drive/MyDrive/tfidf/all_document_combined_keywords.xlsx'
    )

    # Print overall statistics
    print(f"\n{'='*60}")
    print("OVERALL STATISTICS")
    print(f"{'='*60}")

    stats = extractor.get_statistics()
    print(f"📊 Total documents processed: {stats['total_documents']}")
    print(f"📚 Total unigram vocabulary size: {stats['unique_unigrams']}")
    print(f"📚 Total bigram vocabulary size: {stats['unique_bigrams']}")
    print(f"🔗 Terms with suffix mappings: {stats['terms_with_suffix_mappings']}")
    print(f"📝 Average unigrams per document: {stats['avg_unigrams_per_doc']}")
    print(f"📝 Average bigrams per document: {stats['avg_bigrams_per_doc']}")
    print(f"📈 Total unigrams processed: {stats['total_unigrams']}")
    print(f"📈 Total bigrams processed: {stats['total_bigrams']}")

    # Show decision analysis for first document
    if extractor.tfidf_calculator.total_documents > 0:
        print(f"\n🔍 DECISION ANALYSIS SAMPLE (First Document):")
        print("-" * 50)

        decision_analysis = extractor.get_decision_analysis(0)
        if decision_analysis:
            print(f"Document: {decision_analysis['document_name']}")
            print(f"Eliminated unigrams: {decision_analysis['eliminated_unigrams']}")
            print(f"Total keywords: {decision_analysis['total_keywords']}")

            print(f"\nTop 10 PMI Decisions:")
            decisions = decision_analysis['decisions']
            sorted_decisions = sorted(decisions.items(), key=lambda x: abs(x[1]['pmi']), reverse=True)

            for i, (bigram, info) in enumerate(sorted_decisions[:10]):
                print(f"  {i+1:2d}. {bigram:<25} → {info['decision']:<15} (PMI: {info['pmi']:6.2f}) - {info['reason']}")

    # Show most frequently merged terms
    if extractor.vocabulary_mapping:
        print(f"\n🔄 TOP SUFFIX MERGING EXAMPLES:")
        print("-" * 40)
        merged_terms = [(k, v) for k, v in extractor.vocabulary_mapping.items() if len(v) > 1]
        merged_terms.sort(key=lambda x: len(x[1]), reverse=True)

        for normalized, originals in merged_terms[:15]:
            print(f"  {normalized} ← {list(originals)}")

    print(f"\n{'='*60}")
    print("🎉 ENHANCED COMBINED UNIGRAM-BIGRAM ANALYSIS COMPLETE!")
    print("📁 Check the Excel file for detailed results with PMI-based decisions")
    print(f"{'='*60}")

    return extractor

class VideoProcessor:
    def __init__(self, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.device = device
        self.model, self.tokenizer = None, None   # Summary model
        self.model2, self.tokenizer2 = None, None # Title model

        # Always initialize punct_agent
        self.punct_agent = None

        # Initialize punctuation agent
        try:
            from banglanlptoolkit import BanglaPunctuation
            self.punct_agent = BanglaPunctuation()
            print("✅ BanglaPunctuation loaded")
        except ImportError:
            print("⚠️ BanglaPunctuation not available, using fallback")
        except Exception as e:
            print(f"⚠️ Unexpected error loading BanglaPunctuation: {e}")

    def _load_summary_model(self):
        if self.model is None:
            # Free title model if loaded
            if self.model2 is not None:
                del self.model2
                del self.tokenizer2
                self.model2, self.tokenizer2 = None, None
                torch.cuda.empty_cache()

            model_path = "/content/drive/My Drive/Thesis_Dataset/fine_tuned_bangla_t5"
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model.to(self.device)


    def _load_title_model(self):
        if self.model2 is None:
            # Free summary model if loaded
            if self.model is not None:
                del self.model
                del self.tokenizer
                self.model, self.tokenizer = None, None
                torch.cuda.empty_cache()

            self.model2 = AutoModelForSeq2SeqLM.from_pretrained(
                "/content/drive/My Drive/Thesis_Dataset/academic_title_model"
            )
            self.tokenizer2 = AutoTokenizer.from_pretrained(
                "/content/drive/My Drive/Thesis_Dataset/academic_title_model"
            )
            self.model2.to(self.device)



    def safe_normalize(self, text):
        """Safe normalization that works with or without bnunicodenormalizer"""
        try:
            from bnunicodenormalizer import normalize
            return normalize(text)
        except ImportError:
            import unicodedata
            if not text or not isinstance(text, str):
                return text
            text = unicodedata.normalize('NFC', text)
            text = ' '.join(text.split())
            return text.strip()
        except Exception:
            return text

    def extract_audio_from_video(self, video_path, audio_output_path):
        """Extract audio from video file and save as WAV"""
        try:
            print(f"🎬 Loading video: {video_path}")

            if not os.path.exists(video_path):
                raise FileNotFoundError(f"Video file not found: {video_path}")

            video = VideoFileClip(video_path)
            duration = video.duration
            print(f"⏱️ Video duration: {duration/60:.1f} minutes")

            if video.audio is None:
                raise Exception("Video has no audio track!")

            os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)

            print(f"🎵 Extracting audio to: {audio_output_path}")
            video.audio.write_audiofile(
                audio_output_path,
                codec='pcm_s16le',
                verbose=False,
                logger=None
            )

            video.close()

            if os.path.exists(audio_output_path):
                file_size_mb = os.path.getsize(audio_output_path) / (1024 * 1024)
                print(f"✅ Audio extracted successfully! ({file_size_mb:.1f} MB)")
                return True
            else:
                raise Exception("Audio extraction failed - output file not created")

        except Exception as e:
            print(f"❌ Error extracting audio: {e}")
            raise

    def add_punctuation(self, raw_text):
        """Add punctuation to the raw text"""
        if self.punct_agent:
            try:
                raw_text = ' '.join(raw_text.split())
                return self.punct_agent.add_punctuation(raw_text)
            except:
                pass

        # Fallback punctuation logic
        text = raw_text.strip()
        if not text.endswith(('।', '?', '!')):
            text += '।'
        return text

    def format_timestamp(self, ms):
        """Convert milliseconds to MM:SS format"""
        seconds = ms // 1000
        minutes = seconds // 60
        seconds = seconds % 60
        return f"{minutes:02d}:{seconds:02d}"

    def smart_chunk_amplify(self, chunk, max_boost_db=8, target_level_db=-20):
        """
        Enhanced per-chunk amplification with normalization
        - Applies amplification-normalization strategy per chunk
        - Prevents clipping with safety margin
        - Optimizes each chunk individually
        - Maintains consistent loudness across chunks
        """
        # Initial measurements
        current_avg = chunk.dBFS
        peak_level = chunk.max_dBFS

        # Determine desired boost based on average level
        if current_avg > -15:
            # Already loud enough
            desired_boost = 0
        elif current_avg > -25:
            # Moderately quiet
            desired_boost = min(3, max_boost_db)
        elif current_avg > -35:
            # Quiet
            desired_boost = min(6, max_boost_db)
        else:
            # Very quiet
            desired_boost = max_boost_db

        # Apply amplification if needed
        if desired_boost > 0:
            amplified_chunk = chunk + desired_boost
        else:
            amplified_chunk = chunk

        # Normalization step (safety check to prevent clipping)
        if amplified_chunk.max_dBFS > -3:  # Leave 3dB headroom
            normalized_chunk = amplified_chunk.normalize(headroom=3.0)
            return normalized_chunk, desired_boost, current_avg, peak_level, True
        else:
            return amplified_chunk, desired_boost, current_avg, peak_level, False


    def transcribe_audio(self, audio_path, chunk_length_ms=45000, max_tokens=512,
                                        apply_per_chunk_amplification=True, max_boost_db=8):
        """
        Enhanced transcription function with:
        - First: Audio chunking (45 sec) to get sentence-level timestamps
        - Second: Token-based chunking (512 tokens) with proper time mapping
        - Smart per-chunk amplification for better audio quality
        - Overlapping chunks (one sentence overlap)
        """
        # Initialize recognizer with optimized settings
        recognizer = sr.Recognizer()

        # Optimize recognizer settings for better accuracy
        recognizer.energy_threshold = 300
        recognizer.dynamic_energy_threshold = True
        recognizer.pause_threshold = 0.5

        print(f"✅ Optimized speech recognizer initialized")
        print(f"⚡ Per-chunk amplification: {apply_per_chunk_amplification}")
        print(f"🌍 Target language: Bengali (bn-BD)")
        print(f"🎯 Audio chunk size: {chunk_length_ms/1000:.1f} seconds")
        print(f"📝 Token-based final chunks: {max_tokens} tokens max")

        try:
            audio = AudioSegment.from_wav(audio_path)
            print(f"✅ Audio loaded: {len(audio)/1000:.1f} seconds, Avg: {audio.dBFS:.1f} dB, Peak: {audio.max_dBFS:.1f} dB")
        except Exception as e:
            print(f"❌ Error loading audio file: {e}")
            return None

        chunks = make_chunks(audio, chunk_length_ms)
        all_chunks_text = []
        audio_chunks_timestamps = []
        amplification_stats = []
        failed_chunks = 0

        print(f"📝 Step 1: Transcribing audio in {len(chunks)} short chunks for sentence mapping...")

        # Step 1: Transcribe all audio chunks (45 seconds each)
        for i, chunk in enumerate(tqdm(chunks, desc="Transcribing audio chunks")):
            chunk_filename = f"temp_chunk_{i}.wav"

            start_time_ms = i * chunk_length_ms
            end_time_ms = min((i + 1) * chunk_length_ms, len(audio))
            audio_chunks_timestamps.append((start_time_ms, end_time_ms))

            try:
                # Apply smart per-chunk amplification
                if apply_per_chunk_amplification:
                    processed_chunk, boost_applied, orig_avg, orig_peak, is_normalized = self.smart_chunk_amplify(
                        chunk, max_boost_db=max_boost_db
                    )

                    amplification_stats.append({
                        'Chunk_ID': i + 1,
                        'Original_Avg_dB': round(orig_avg, 1),
                        'Original_Peak_dB': round(orig_peak, 1),
                        'Boost_Applied_dB': round(boost_applied, 1),
                        'Final_Avg_dB': round(processed_chunk.dBFS, 1),
                        'Final_Peak_dB': round(processed_chunk.max_dBFS, 1),
                        'Was_Normalized': is_normalized
                    })
                else:
                    processed_chunk = chunk
                    boost_applied = 0

                processed_chunk.export(chunk_filename, format="wav")

                with sr.AudioFile(chunk_filename) as source:
                    audio_data = recognizer.record(source)
                    try:
                        text = recognizer.recognize_google(audio_data, language="bn-BD")
                        text_clean = ' '.join(text.split())
                        punctuated_text = self.add_punctuation(text_clean)
                        all_chunks_text.append(punctuated_text)
                    except sr.UnknownValueError:
                        all_chunks_text.append("[Unrecognized Audio]")
                        failed_chunks += 1
                    except sr.RequestError as e:
                        all_chunks_text.append(f"[Request Error: {e}]")
                        failed_chunks += 1

                if os.path.exists(chunk_filename):
                    os.remove(chunk_filename)

            except Exception as e:
                print(f"Error processing chunk {i}: {e}")
                all_chunks_text.append(f"[Processing Error: {e}]")
                failed_chunks += 1
                if os.path.exists(chunk_filename):
                    os.remove(chunk_filename)

        # Step 2: Create sentence-level mapping with timestamps and carry-over handling
        print("🔄 Step 2: Creating sentence-level timestamp mapping with carry-over handling...")

        sentence_data = []
        sentence_id = 1
        carry_over_text = ""  # For incomplete sentences across audio chunks
        carry_over_start_ms = 0

        for chunk_idx, (text, (start_ms, end_ms)) in enumerate(zip(all_chunks_text, audio_chunks_timestamps)):
            if text.startswith("["):
                # Error chunk - treat as one sentence, but handle carry-over first
                if carry_over_text.strip():
                    # Save carry-over as incomplete sentence before error chunk
                    sentence_data.append({
                        'sentence_id': sentence_id,
                        'text': carry_over_text.strip(),
                        'start_ms': carry_over_start_ms,
                        'end_ms': start_ms,  # End at start of error chunk
                        'chunk_duration_ms': start_ms - carry_over_start_ms,
                        'source_chunk_id': chunk_idx,
                        'is_error': False,
                        'is_complete': False
                    })
                    sentence_id += 1
                    carry_over_text = ""

                # Add error chunk
                sentence_data.append({
                    'sentence_id': sentence_id,
                    'text': text,
                    'start_ms': start_ms,
                    'end_ms': end_ms,
                    'chunk_duration_ms': end_ms - start_ms,
                    'source_chunk_id': chunk_idx + 1,
                    'is_error': True,
                    'is_complete': True
                })
                sentence_id += 1
                continue

            # Combine carry-over text with current chunk
            full_text = carry_over_text + " " + text if carry_over_text else text
            full_text = full_text.strip()

            # Split into sentences
            sentences = split_into_sentences(full_text)
            if not sentences:
                # No sentences found, treat entire text as carry-over
                if not carry_over_text:
                    carry_over_start_ms = start_ms
                carry_over_text = full_text
                continue

            # Check if the last sentence is complete
            last_sentence_complete = False
            if full_text.strip():
                if re.search(r'[।?!]\s*$', full_text.strip()):
                    last_sentence_complete = True

            if last_sentence_complete or chunk_idx == len(all_chunks_text) - 1:
                complete_sentences = sentences
                new_carry_over = ""
            else:
                if len(sentences) > 1:
                    complete_sentences = sentences[:-1]
                    new_carry_over = sentences[-1] if sentences else ""
                else:
                    complete_sentences = []
                    new_carry_over = sentences[0] if sentences else full_text

            if complete_sentences:
                chunk_duration_ms = end_ms - start_ms
                actual_start_ms = carry_over_start_ms if carry_over_text else start_ms

                if carry_over_text:
                    total_duration_ms = end_ms - carry_over_start_ms
                    time_per_sentence = total_duration_ms / len(complete_sentences) if complete_sentences else total_duration_ms
                else:
                    time_per_sentence = chunk_duration_ms / len(complete_sentences) if complete_sentences else chunk_duration_ms

                for sent_idx, sentence in enumerate(complete_sentences):
                    if sentence.strip():
                        sent_start_ms = actual_start_ms + (sent_idx * time_per_sentence)
                        sent_end_ms = actual_start_ms + ((sent_idx + 1) * time_per_sentence)

                        sentence_data.append({
                            'sentence_id': sentence_id,
                            'text': sentence.strip(),
                            'start_ms': int(sent_start_ms),
                            'end_ms': int(sent_end_ms),
                            'chunk_duration_ms': int(sent_end_ms - sent_start_ms),
                            'source_chunk_id': chunk_idx + 1,
                            'is_error': False,
                            'is_complete': True
                        })
                        sentence_id += 1

            carry_over_text = new_carry_over
            if carry_over_text:
                if complete_sentences:
                    carry_over_start_ms = int(actual_start_ms + (len(complete_sentences) * time_per_sentence))
                else:
                    carry_over_start_ms = start_ms

        if carry_over_text.strip():
            last_end_ms = audio_chunks_timestamps[-1][1] if audio_chunks_timestamps else carry_over_start_ms
            sentence_data.append({
                'sentence_id': sentence_id,
                'text': carry_over_text.strip(),
                'start_ms': carry_over_start_ms,
                'end_ms': last_end_ms,
                'chunk_duration_ms': last_end_ms - carry_over_start_ms,
                'source_chunk_id': len(all_chunks_text),
                'is_error': False,
                'is_complete': False
            })
        print(f"✅ Created {len(sentence_data)} sentences with timestamps")

        # Step 3: Create token-based chunks with overlapping and carry-over handling
        print(f"🔄 Step 3: Creating token-based chunks (max {max_tokens} tokens) with overlap and carry-over...")

        def count_tokens(text):
            try:
                tokens = self.tokenizer.encode(text)
                return len(tokens)
            except:
                return len(text) // 4

        token_based_chunks = []
        current_chunk_sentences = []
        current_chunk_tokens = 0
        previous_last_sentence = None
        incomplete_sentence_buffer = []
        max_tokens = 512

        for sentence in sentence_data:
            sentence_text = sentence['text']
            sentence_tokens = count_tokens(sentence_text)
            is_complete = sentence.get('is_complete', True)

            if not is_complete and not sentence.get('is_error', False):
                incomplete_sentence_buffer.append(sentence)
                continue

            if incomplete_sentence_buffer:
                merged_text_parts = [s['text'] for s in incomplete_sentence_buffer] + [sentence_text]
                merged_text = ' '.join(merged_text_parts).strip()

                merged_sentence = {
                    'sentence_id': incomplete_sentence_buffer[0]['sentence_id'],
                    'text': merged_text,
                    'start_ms': incomplete_sentence_buffer[0]['start_ms'],
                    'end_ms': sentence['end_ms'],
                    'is_complete': True,
                    'is_error': sentence.get('is_error', False),
                    'merged_from_count': len(incomplete_sentence_buffer) + 1
                }

                incomplete_sentence_buffer = []
                sentence_to_process = merged_sentence
                sentence_tokens = count_tokens(merged_text)
            else:
                sentence_to_process = sentence

            overlap_tokens = 0
            if previous_last_sentence is not None:
                overlap_tokens = count_tokens(previous_last_sentence['text'])

            total_tokens_needed = current_chunk_tokens + sentence_tokens + overlap_tokens

            if (total_tokens_needed > max_tokens and current_chunk_sentences) or (sentence_tokens + overlap_tokens > max_tokens):
                if current_chunk_sentences:
                    chunk_sentences_with_overlap = []
                    has_overlap = False
                    overlap_sentence_text = ""
                    actual_overlap_tokens = 0

                    if previous_last_sentence is not None and len(token_based_chunks) > 0:
                        overlap_text = previous_last_sentence['text']
                        actual_overlap_tokens = count_tokens(overlap_text)

                        if current_chunk_tokens + actual_overlap_tokens <= max_tokens:
                            overlap_sentence_dict = {
                                'text': overlap_text,
                                'sentence_id': previous_last_sentence['sentence_id'],
                                'start_ms': previous_last_sentence['start_ms'],
                                'end_ms': previous_last_sentence['end_ms']
                            }
                            chunk_sentences_with_overlap.append(overlap_sentence_dict)
                            has_overlap = True
                            overlap_sentence_text = overlap_text

                    chunk_sentences_with_overlap.extend(current_chunk_sentences)

                    chunk_start_time = chunk_sentences_with_overlap[0]['start_ms']
                    chunk_end_time = chunk_sentences_with_overlap[-1]['end_ms']
                    chunk_text = '।'.join([s['text'] for s in chunk_sentences_with_overlap]) + '।'

                    final_token_count = count_tokens(chunk_text)

                    token_based_chunks.append({
                        'text': chunk_text,
                        'start_ms': chunk_start_time,
                        'end_ms': chunk_end_time,
                        'token_count': final_token_count,
                        'sentence_count': len(current_chunk_sentences),
                        'first_sentence_id': current_chunk_sentences[0]['sentence_id'],
                        'last_sentence_id': current_chunk_sentences[-1]['sentence_id'],
                        'has_overlap': has_overlap,
                        'overlap_sentence': overlap_sentence_text,
                        'overlap_tokens': actual_overlap_tokens,
                        'base_tokens': current_chunk_tokens,
                        'has_merged_sentences': any(s.get('merged_from_count', 0) > 1 for s in current_chunk_sentences)
                    })

                    previous_last_sentence = current_chunk_sentences[-1]

                current_chunk_sentences = [sentence_to_process]
                current_chunk_tokens = sentence_tokens

                if sentence_tokens + overlap_tokens > max_tokens:
                    print(f"⚠️ Warning: Sentence {sentence_to_process.get('sentence_id', 'unknown')} with overlap exceeds max_tokens ({sentence_tokens + overlap_tokens} > {max_tokens})")
            else:
                current_chunk_sentences.append(sentence_to_process)
                current_chunk_tokens += sentence_tokens

        if incomplete_sentence_buffer:
            merged_text = ' '.join([s['text'] for s in incomplete_sentence_buffer]).strip()
            merged_sentence = {
                'sentence_id': incomplete_sentence_buffer[0]['sentence_id'],
                'text': merged_text,
                'start_ms': incomplete_sentence_buffer[0]['start_ms'],
                'end_ms': incomplete_sentence_buffer[-1]['end_ms'],
                'is_complete': False,
                'is_error': False,
                'merged_from_count': len(incomplete_sentence_buffer)
            }
            current_chunk_sentences.append(merged_sentence)
            current_chunk_tokens += count_tokens(merged_text)

        if current_chunk_sentences:
            chunk_sentences_with_overlap = []
            has_overlap = False
            overlap_sentence_text = ""
            actual_overlap_tokens = 0

            if previous_last_sentence is not None and len(token_based_chunks) > 0:
                overlap_text = previous_last_sentence['text']
                actual_overlap_tokens = count_tokens(overlap_text)

                if current_chunk_tokens + actual_overlap_tokens <= max_tokens:
                    overlap_sentence_dict = {
                        'text': overlap_text,
                        'sentence_id': previous_last_sentence['sentence_id'],
                        'start_ms': previous_last_sentence['start_ms'],
                        'end_ms': previous_last_sentence['end_ms']
                    }
                    chunk_sentences_with_overlap.append(overlap_sentence_dict)
                    has_overlap = True
                    overlap_sentence_text = overlap_text

            chunk_sentences_with_overlap.extend(current_chunk_sentences)

            chunk_start_time = chunk_sentences_with_overlap[0]['start_ms']
            chunk_end_time = chunk_sentences_with_overlap[-1]['end_ms']
            chunk_text = '।'.join([s['text'] for s in chunk_sentences_with_overlap]) + '।'

            final_token_count = count_tokens(chunk_text)

            token_based_chunks.append({
                'text': chunk_text,
                'start_ms': chunk_start_time,
                'end_ms': chunk_end_time,
                'token_count': final_token_count,
                'sentence_count': len(current_chunk_sentences),
                'first_sentence_id': current_chunk_sentences[0]['sentence_id'],
                'last_sentence_id': current_chunk_sentences[-1]['sentence_id'],
                'has_overlap': has_overlap,
                'overlap_sentence': overlap_sentence_text,
                'overlap_tokens': actual_overlap_tokens,
                'base_tokens': current_chunk_tokens,
                'has_merged_sentences': any(s.get('merged_from_count', 0) > 1 for s in current_chunk_sentences)
            })

        print(f"✅ Created {len(token_based_chunks)} token-based chunks")

        # Step 4: Create DataFrames
        print("📊 Step 4: Preparing output data...")

        token_chunk_data = []
        for i, chunk in enumerate(token_based_chunks):
            token_chunk_data.append({
                "Chunk_ID": i + 1,
                "Start_Time": self.format_timestamp(chunk["start_ms"]),
                "End_Time": self.format_timestamp(chunk["end_ms"]),
                "Duration_MS": chunk["end_ms"] - chunk["start_ms"],
                "Text": chunk["text"],
                "Token_Count": chunk["token_count"],
                "Sentence_Count": chunk["sentence_count"],
                "First_Sentence_ID": chunk["first_sentence_id"],
                "Last_Sentence_ID": chunk["last_sentence_id"],
                "Has_Overlap": chunk["has_overlap"],
                "Overlap_Sentence": chunk["overlap_sentence"],
                "Overlap_Tokens": chunk.get("overlap_tokens", 0),
                "Base_Tokens": chunk.get("base_tokens", 0),
                "Has_Merged_Sentences": chunk.get("has_merged_sentences", False)
            })




        return pd.DataFrame(token_chunk_data)


    def generate_summary(self, text):
        try:
            self._load_summary_model()  # ensure summary model is loaded

            input_text = f"summarize: {text}"
            inputs = self.tokenizer(
                input_text,
                max_length=512,
                truncation=True,
                return_tensors="pt",
                padding="max_length"
            ).to(self.device)

            with torch.no_grad():
                summary_ids = self.model.generate(
                    inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_length=200,
                    min_length=100,
                    length_penalty=1.0,
                    num_beams=4,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    do_sample=False
                )

            summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            return summary if summary.strip() else text[:200] + "..."
        except Exception as e:
            print(f"Error generating summary: {e}")
            return text[:200] + "..." if len(text) > 200 else text

    def generate_title(self, summary):
        self._load_title_model()  # ensure title model is loaded

        # Handle long summaries
        tokens = self.tokenizer2.encode(summary, add_special_tokens=False)
        if len(tokens) > 1024:
            tokens = tokens[:1024]
            summary = self.tokenizer2.decode(tokens, skip_special_tokens=True)

        inputs = self.tokenizer2(summary, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            title_ids = self.model2.generate(inputs['input_ids'], num_beams=4, min_length=5, max_length=30)

        return self.tokenizer2.decode(title_ids[0], skip_special_tokens=True)

    def concatenate_pairs(self, df):
        """
        Concatenate pairs of rows in the dataframe
        - Combines text/summaries from two consecutive rows
        - Updates Start_Time and End_Time accordingly
        """
        new_rows = []

        for i in range(0, len(df), 2):
            if i + 1 < len(df):
                # Pair exists - concatenate two rows
                row1 = df.iloc[i]
                row2 = df.iloc[i + 1]

                # Combine text (use Summary if available, otherwise Text)
                text1 = row1.get('Summary', row1.get('Text', ''))
                text2 = row2.get('Summary', row2.get('Text', ''))
                combined_text = f"{text1} {text2}"

                # Create new row with combined data
                new_row = {
                    'Chunk_ID': f"combined_{i//2 + 1}",
                    'Text': combined_text,
                    'Start_Time': row1.get('Start_Time', ''),
                    'End_Time': row2.get('End_Time', '')
                }

                # Preserve any other columns from the first row
                for col in df.columns:
                    if col not in ['Chunk_ID', 'Text', 'Start_Time', 'End_Time', 'Summary']:
                        new_row[col] = row1.get(col, '')

                new_rows.append(new_row)
            else:
                # Odd row - keep as is
                row = df.iloc[i].copy()
                row['Chunk_ID'] = f"single_{i//2 + 1}"

                # Use Summary if available, otherwise Text
                if 'Summary' in row and pd.notna(row['Summary']):
                    row['Text'] = row['Summary']

                new_rows.append(row.to_dict())

        return pd.DataFrame(new_rows)

    # Function to count tokens
    def count_tokens(self, text, tokenizer):
        return len(tokenizer.encode(text, truncation=False, padding=False))

    def word_count(self,text):
      if pd.isna(text):
        return 0
      return len(str(text).split())

    def clean_text(self,text):
      if pd.isna(text):
        return ""
      return re.sub(r"\[[a-zA-Z _]+\]", " ", str(text))





    def process_complete_pipeline(self, video_file_path, temp_dir, max_chunks=50, job_id=None,top_k=20):
        try:
            # File paths
            audio_path = os.path.join(temp_dir, "extracted_audio.wav")
            transcription_path = os.path.join(temp_dir, "transcription.xlsx")
            summary_path = os.path.join(temp_dir, "summaries.xlsx")
            final_text_path = os.path.join(temp_dir, "final_summary.txt")

            def update_status(progress, message):
                """Helper function to update both local results and global status"""
                if job_id and job_id in processing_status:
                    processing_status[job_id].update({
                        'progress': progress,
                        'message': message,
                        'status': 'processing'
                    })

            # Step 1: Extract audio
            update_status(10, 'Extracting audio from video...')
            self.extract_audio_from_video(video_file_path, audio_path)

            # Step 2: Transcribe
            update_status(30, 'Transcribing audio...')
            transcription_df = self.transcribe_audio(audio_path)
            transcription_df["Text"] = transcription_df["Text"].apply(self.clean_text)

            # Save transcription
            transcription_df.to_excel(transcription_path, index=False)

            # Step 3: Generate summaries with recursive logic
            update_status(50, 'Processing summaries...')

            # Check if we need recursive summarization
            if len(transcription_df) > max_chunks:
                update_status(55, f'Input has {len(transcription_df)} chunks (>{max_chunks}). Starting recursive summarization...')
                print(f"⚡ Input has {len(transcription_df)} rows (>{max_chunks}). Starting recursive summarization...")

                # Perform recursive summarization
                final_df, total_iterations = self.recursive_summarization_with_concatenation(
                    transcription_df, max_chunks, job_id
                )

                update_status(75, f'Completed recursive summarization in {total_iterations} iterations')
                print(f"🎯 Completed recursive summarization in {total_iterations} iterations")

            else:
                update_status(60, f'Input has ≤{max_chunks} chunks. Generating summaries directly...')
                print(f"📄 Input has ≤{max_chunks} rows. Generating summaries directly...")

                # Direct summarization for smaller datasets
                transcription_df['Final_Summary'] = transcription_df['Text'].apply(
                    lambda x: self.generate_summary(str(x)) if not str(x).startswith('[') else x
                )
                final_df = transcription_df
                total_iterations = 1

            # Step 4: Save final summaries
            update_status(80, 'Saving summary results...')

            # Ensure Final_Summary column exists
            if 'Final_Summary' not in final_df.columns:
                final_df['Final_Summary'] = final_df['Text'].apply(
                    lambda x: self.generate_summary(str(x)) if not str(x).startswith('[') else x
                )

            # Save summaries
            final_df.to_excel(summary_path, index=False)
            print(f"DataFrame shape: {final_df.shape}")
            print(f"Number of rows: {len(final_df)}")
            total_text_words = final_df["Text"].apply(self.word_count).sum()
            total_summary_words = final_df["Final_Summary"].apply(self.word_count).sum()
            compression_ratio = total_summary_words / total_text_words if total_text_words > 0 else 0
            # Step 5: Create final text with timestamps
            update_status(90, 'Creating final summary document...')

            tmp_text=""
            for index, row in final_df.iterrows():

                summary = row.get('Final_Summary', row.get('Text', ''))
                tmp_text += f"{summary}"



            TARGET_FOLDER = "/content/drive/MyDrive/tfidf/Concatenated_Summary"
            EXCEL_PATH = "/content/drive/MyDrive/tfidf/all_document_combined_keywords.xlsx"

            tmp_text=tmp_text.strip()
            title=self.generate_title(tmp_text)
            SOURCE_FILE_PATH="/content/drive/MyDrive/tfidf/new.txt"

            with open(SOURCE_FILE_PATH, 'w', encoding='utf-8') as file:
              file.write(tmp_text)  # Or tmp_text if that's what you're using

            print(f"File saved successfully to: {SOURCE_FILE_PATH}")

            source_filename = os.path.basename(SOURCE_FILE_PATH)
            target_file_path = os.path.join(TARGET_FOLDER, source_filename)

            try:
              shutil.copy2(SOURCE_FILE_PATH, target_file_path)
              print(f"✅ File copied to: {target_file_path}")

            except Exception as e:
              print(f"❌ Error copying file: {e}")
              exit()

            try:
              extractor = enhanced_combined_main()  # This will process the TARGET_FOLDER
              if extractor is None:
                print("❌ enhanced_combined_main() failed")
                exit()
            except Exception as e:
              print(f"❌ Error in enhanced_combined_main(): {e}")
              exit()







            try:
              if not os.path.exists(EXCEL_PATH):
                print(f"❌ Excel file not found: {EXCEL_PATH}")
                exit()

              df = pd.read_excel(EXCEL_PATH)
              target_filename = source_filename  # This already has extension like doc1.txt
              matching_rows = df[df['Document_Name'] == target_filename]
              if matching_rows.empty:
                unique_docs = df['Document_Name'].unique()
                for i, doc in enumerate(unique_docs[:10]):
                  print(f"   {i+1}. {doc}")
                if len(unique_docs) > 10:
                  print(f"   ... and {len(unique_docs) - 10} more")
              else:
                matching_rows = matching_rows.sort_values('Rank')
                keywords = matching_rows['Keyword'].tolist()
                print("Extracted Keywords:")
                for keyword in keywords:
                  print(keyword)
                doc_name = os.path.splitext(target_filename)[0]
                output_filename = f"{doc_name}_combined_keywords.xlsx"
                output_path = os.path.join("/content/drive/MyDrive/tfidf/", output_filename)
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                  matching_rows.to_excel(writer, sheet_name='Keywords', index=False)
                  decision_summary = matching_rows.groupby(['Term_Type', 'Decision']).size().reset_index(name='Count')
                  decision_summary.to_excel(writer, sheet_name='Decision_Summary', index=False)
                  pmi_analysis = matching_rows[matching_rows['PMI'] != 0].sort_values('PMI', ascending=False)
                  if not pmi_analysis.empty:
                    pmi_analysis.to_excel(writer, sheet_name='PMI_Analysis', index=False)
                print(f"💾 Combined keywords saved to: {output_path}")
            except Exception as e:
              print(f"❌ Error extracting keywords: {e}")
            try:
              os.remove(target_file_path)
            except Exception as e:
              print(f"⚠️  Warning: Could not remove file: {e}")

            keywords_text = ", ".join(keywords)
            final_text = f"Title: {title}\n\n"
            final_text += f"Important Keywords: {keywords_text}\n\n"
            final_text += f"Compression Ratio: {compression_ratio:.4f}\n\n"
            for index, row in final_df.iterrows():
                start_time = row.get('Start_Time', '')
                end_time = row.get('End_Time', '')
                summary = row.get('Final_Summary', row.get('Text', ''))
                chunk = f"[{start_time} - {end_time}]\n{summary}\n\n"
                print(f"Row {index}: {repr(chunk)}")

                final_text += chunk


            # 2. Define a dictionary of regex patterns and replacements
                regex_patterns = {
    r"টু দি (পাওয়ার|পাওয়ার)|টু দা (পাওয়ার|পাওয়ার)": "^",     # Replace "টু দি পাওয়ার" with "^"
    r"সমান সমান|ইকুয়ালস টু|ইকুয়ালস টু|ইকুয়াল টু|ইকুয়াল টু": "=",

                 # Replace "দশ" and "টেন" with "10"
    r"এগারো|এলেভেন": "11",      # Replace "এগারো" and "এলেভেন" with "11"
    r"বারো|টুয়েলভ|টুয়েলভ": "12",        # Replace "বারো" and "টুয়েলভ" with "12"
    r"তেরো|থার্টিন": "13",      # Replace "তেরো" and "থার্টিন" with "13"
    r"চোদ্দো|ফোর্টিন": "14",    # Replace "চোদ্দো" and "ফোর্টিন" with "14"
    r"পনেরো|ফিফটিন": "15",      # Replace "পনেরো" and "ফিফটিন" with "15"
    r"ষোলো|সিক্সটিন": "16",     # Replace "ষোলো" and "সিক্সটিন" with "16"
    r"সতেরো|সেভেন্টিন": "17",   # Replace "সতেরো" and "সেভেন্টিন" with "17"
    r"আঠারো|এইটিন": "18",       # Replace "আঠারো" and "এইটিন" with "18"
    r"ঊনিশ|নাইনটিন": "19",       # Replace "ঊনিশ" and "নিনটিন" with "19"

    r"একুশ|(টুয়েন্টি|টুয়েন্টি) (ওয়ান|ওয়ান)": "21",# Replace "একুশ" and "টুয়েন্টি ওয়ান" with "21"
    r"বাইশ|(টুয়েন্টি|টুয়েন্টি) টু": "22",  # Replace "বাইশ" and "টুয়েন্টি টু" with "22"
    r"তেইশ|(টুয়েন্টি|টুয়েন্টি) থ্রি": "23",# Replace "তেইশ" and "টুয়েন্টি থ্রি" with "23"
    r"চব্বিশ|(টুয়েন্টি|টুয়েন্টি) ফোর": "24",# Replace "চব্বিশ" and "টুয়েন্টি ফোর" with "24"
    r"পঁচিশ|(টুয়েন্টি|টুয়েন্টি) ফাইভ": "25",# Replace "পঁইত্রিশ" and "থার্টি ফাইভ" with "25"
    r"ছাব্বিশ|(টুয়েন্টি|টুয়েন্টি) সিক্স": "26",# Replace "ছত্রিশ" and "থার্টি সিক্স" with "26"
    r"সাতাশ|(টুয়েন্টি|টুয়েন্টি) সেভেন": "27",# Replace "সাতত্রিশ" and "থার্টি সেভেন" with "27"
    r"আটাশ|(টুয়েন্টি|টুয়েন্টি) এইট": "28",# Replace "আটত্রিশ" and "থার্টি এইট" with "28"
    r"ঊনত্রিশ|(টুয়েন্টি|টুয়েন্টি) নাইন": "29",# Replace "ঊনচল্লিশ" and "থার্টি নাইন" with "29"

    r"একত্রিশ|থার্টি (ওয়ান|ওয়ান)": "31",# Replace "একত্রিশ" and "থার্টি ওয়ান" with "31"
    r"বত্রিশ|থার্টি টু": "32",   # Replace "বত্রিশ" and "থার্টি টু" with "32"
    r"তেত্রিশ|থার্টি থ্রি": "33",# Replace "তেত্রিশ" and "থার্টি থ্রি" with "33"
    r"চৌত্রিশ|থার্টি ফোর": "34",# Replace "চত্রিশ" and "থার্টি ফোর" with "34"
    r"পঁইয়ত্রিশ|পঁইয়ত্রিশ|থার্টি ফাইভ": "35",# Replace "পঁইত্রিশ" and "থার্টি ফাইভ" with "35"
    r"ছত্রিশ|থার্টি সিক্স": "36",# Replace "ছত্রিশ" and "থার্টি সিক্স" with "36"
    r"সাইত্রিশ|থার্টি সেভেন": "37",# Replace "সাতত্রিশ" and "থার্টি সেভেন" with "37"
    r"আটত্রিশ|থার্টি এইট": "38",# Replace "আটত্রিশ" and "থার্টি এইট" with "38"
    r"ঊনচল্লিশ|থার্টি নাইন": "39",# Replace "ঊনপঁইত্রিশ" and "থার্টি নাইন" with "39"

    r"একচল্লিশ|ফরটি (ওয়ান|ওয়ান)": "41",# Replace "একতল্লিশ" and "ফরটি ওয়ান" with "41"
    r"বিয়াল্লিশ|বিয়াল্লিশ|ফরটি টু": "42",# Replace "বিয়াল্লিশ" and "ফরটি টু" with "42"
    r"তেতাল্লিশ|ফরটি থ্রি": "43",# Replace "তেতাল্লিশ" and "ফরটি থ্রি" with "43"
    r"চুয়াল্লিশ|চুয়াল্লিশ|ফরটি ফোর": "44",# Replace "চুয়াল্লিশ" and "ফরটি ফোর" with "44"
    r"পঁয়তাল্লিশ|পঁয়তাল্লিশ|ফরটি ফাইভ": "45",# Replace "পঁইত্রাল্লিশ" and "ফরটি ফাইভ" with "45"
    r"ছেচল্লিশ|ফরটি সিক্স": "46",     # Replace "ছেষট্টি" and "ফিফটি" with "46"
    r"সাতচল্লিশ|ফরটি সেভেন": "47",# Replace "সাতষট্টি" and "ফিফটি ওয়ান" with "47"
    r"আটচল্লিশ|ফরটি এইট": "48",  # Replace "আটষট্টি" and "ফিফটি টু" with "48"
    r"ঊনপঞ্চাশ|ফরটি নাইন": "49",# Replace "ঊননব্বই" and "ফিফটি থ্রি" with "49"

    r"একান্ন|ফিফটি (ওয়ান|ওয়ান)": "51", # Replace "একান্ন" and "ফিফটি ফাইভ" with "51"
    r"বায়ান্ন|ফিফটি টু": "52",# Replace "বায়ান্ন" and "ফিফটি সিক্স" with "52"
    r"তিপ্পান্ন|ফিফটি থ্রি": "53",# Replace "তিপ্পান্ন" and "ফিফটি সেভেন" with "53"
    r"চুয়ান্ন|চুয়ান্ন|ফিফটি ফোর": "54",# Replace "চুয়ান্ন" and "ফিফটি এইট" with "54"
    r"পঞ্চান্ন|ফিফটি ফাইভ": "55",# Replace "পঁইপঁচান্ন" and "ফিফটি নাইন" with "55"
    r"ছাপ্পান্ন|ফিফটি সিক্স": "56",
    r"সাতান্ন|ফিফটি সেভেন": "57",
    r"এটান্ন|ফিফটি এইট": "58",
    r"ঊনষাট|ফিফটি নাইন": "59",

    r"একষট্টি|সিক্সটি (ওয়ান|ওয়ান)": "61",    # Replace "একষট্টি" and "সেভেনটি" with "61"
    r"বাষট্টি|সিক্সটি টু": "62",# Replace "বাষট্টি" and "সেভেনটি ওয়ান" with "62"
    r"তেষট্টি|সিক্সটি থ্রি": "63",
    r"চৌষট্টি|সিক্সটি ফোর": "64",
    r"পঁইষট্টি|সিক্সটি ফাইভ": "65",
    r"ছেষট্টি|সিক্সটি সিক্স": "66",
    r"সাতষট্টি|সিক্সটি সেভেন": "67",
    r"আটষট্টি|সিক্সটি এইট": "68",
    r"ঊনসত্তর|সিক্সটি নাইন": "69",

    r"একাত্তর|সেভেন্টি (ওয়ান|ওয়ান)": "71",
    r"বাহাত্তর|সেভেন্টি টু।বায়াত্তর|বিয়াত্তর": "72",
    r"তিয়াত্তর|তিয়াত্তর|সেভেন্টি থ্রি": "73",
    r"চুয়াত্তর|চুয়াত্তর|সেভেন্টি ফোর": "74",
    r"পঁচাত্তর|সেভেন্টি ফাইভ": "75",
    r"ছিয়াত্তর|ছিয়াত্তর|সেভেন্টি সিক্স": "76",
    r"সাতাত্তর|সেভেন্টি সেভেন": "77",
    r"আটাত্তর|সেভেন্টি এইট": "78",
    r"ঊনআশি|সেভেন্টি নাইন": "79",

    r"একাশি|এইটি (ওয়ান|ওয়ান)": "81",
    r"বিরাশি|এইটি টু": "82",
    r"তিরাশি|এইটি থ্রি": "83",
    r"চুরাশি|এইটি ফোর": "84",
    r"পঁচাশি|এইটি ফাইভ": "85",
    r"ছিয়াশি|ছিয়াশি|এইটি সিক্স": "86",
    r"সাতাশি|এইটি সেভেন": "87",
    r"আটাশি|এইটি এইট": "88",
    r"ঊননব্বই|এইটি নাইন": "89",

    r"একানব্বই|নাইনটি (ওয়ান|ওয়ান)": "91",
    r"বিরানব্বই|নাইনটি টু": "92",
    r"তিরানব্বই|নাইনটি থ্রি": "93",
    r"চুরানব্বই|নাইনটি ফোর": "94",
    r"পঁচানব্বই|নাইনটি ফাইভ": "95",
    r"ছিয়ানব্বই|ছিয়ানব্বই|নাইনটি সিক্স": "96",
    r"সাতানব্বই|নাইনটি সেভেন": "97",
    r"আটানব্বই|নাইনটি এইট": "98",
    r"নিরানব্বই|নাইনটি নাইন": "99",

             # Replace "বিশ" and "টুইন্টি" with "20"
    r"ত্রিশ|থার্টি": "30",      # Replace "ত্রিশ" and "থার্টি" with "30"
     r"চল্লিশ|ফরটি": "40",      # Replace "চল্লিশ" and "ফরটি" with "40"
     r"পঞ্চাশ|ফিফটি": "50",  # Replace "পঞ্চাশ" and "ফিফটি ফোর" with "50"
     r"ষাট|সিক্সটি": "60",        # Replace "ষাট" and "সিক্সটি" with "60"
    r"সত্তর|সেভেন্টি": "70",
     r"নব্বই|নাইনটি": "90",

     r"ওয়ান|ওয়ান": "1" ,      # Replace "ওয়ান", "এক", and "থ্রি" with "1"
     r" টু।": " 2।",
     r" টু\)": " 2)",
     r"দুই": "2",      # Replace "টু", "দুই", "two" with "2"
     r" টু ": " 2 ",
     r" টু,": " 2,",
     r"\(টু ": "(2 ",

    r" তিন | থ্রি ": " 3 ",            # Replace "তিন" and "থ্রি" with "3"
    r" চার | ফোর ": " 4 ",             # Replace "চার" and "ফোর" with "4"
    r" পাঁচ | ফাইভ ": " 5 ",           # Replace "পাঁচ" and "ফাইভ" with "5"
    r" ছয়| ছয় | সিক্স ": " 6 ",            # Replace "ছয়" and "সিক্স" with "6"
    r" সাত | সেভেন ": " 7 ",           # Replace "সাত" and "সেভেন" with "7"
    r" আট | এইট ": " 8 ",              # Replace "আট" and "এইট" with "8"
    r" নাইন ": " 9 ",             # Replace "নয়" and "নাইন" with "9"

    r" তিন,| থ্রি,": " 3,",
    r" তিন।| থ্রি।": " 3।",
    r" তিন\)| থ্রি\)": " 3)",
    r"^তিন |^থ্রি ": "3 ",
    r"\(তিন |\(থ্রি ": "(3 ",

    # 4
    r" চার,| ফোর,": " 4,",
    r" চার।| ফোর।": " 4।",
    r" চার\)| ফোর\)": " 4)",
    r"^চার |^ফোর ": "4 ",
    r"\(চার |\(ফোর ": "(4 ",

    # 5
    r" পাঁচ,| ফাইভ,": " 5,",
    r" পাঁচ।| ফাইভ।": " 5।",
    r" পাঁচ\)| ফাইভ\)": " 5)",
    r"^পাঁচ |^ফাইভ ": "5 ",
    r"\(পাঁচ |\(ফাইভ ": "(5 ",

    # 6
    r" (ছয়|ছয়),| সিক্স,": " 6,",
    r" (ছয়|ছয়)।| সিক্স।": " 6।",
    r" (ছয়|ছয়)\)| সিক্স\)": " 6)",
    r"^(ছয়|ছয়) |^সিক্স ": "6 ",
    r"\((ছয়|ছয়) |\(সিক্স ": "(6 ",

    # 7
    r" সাত,| সেভেন,": " 7,",
    r" সাত।| সেভেন।": " 7।",
    r" সাত\)| সেভেন\)": " 7)",
    r"^সাত |^সেভেন ": "7 ",
    r"\(সাত |\(সেভেন ": "(7 ",

    # 8
    r" আট,| এইট,": " 8,",
    r" আট।| এইট।": " 8।",
    r" আট\)| এইট\)": " 8)",
    r"^আট |^এইট ": "8 ",
    r"\(আট |\(এইট ": "(8 ",

    # 9
    r" নাইন,": " 9,",
    r" নাইন।": " 9।",
    r" নয়\)| নয়\)| নাইন\)": " 9)",
    r"^নয় |^নয় |^নাইন ": "9 ",
    r"\(নয় |\(নয় |\(নাইন ": "(9 ",

    # 10
   r" দশ,| টেন,": " 10,",
   r" দশ।| টেন।": " 10।",
   r" দশ\)| টেন\)": " 10)",
   r"^দশ |^টেন ": "10 ",
   r" দশ | টেন ": " 10 ",
   r"\(দশ |\(টেন ": "(10 ",

  # 20
    r"টুয়েন্টি|টুয়েন্টি": "20",
  r" বিশ,| টুইন্টি,": " 20,",
  r" বিশ।| টুইন্টি।": " 20।",
  r" বিশ\)| টুইন্টি\)": " 20)",
  r"^বিশ |^টুইন্টি ": "20 ",
  r" বিশ | টুইন্টি ": " 20 ",
  r"\(বিশ |\(টুইন্টি ": "(20 ",

    # 80
    r" আশি,| এইটি,": " 80,",
    r" আশি।| এইটি।": " 80।",
    r" আশি\)| এইটি\)": " 80)",
    r"^আশি |^এইটি ": "80 ",
    r" আশি | এইটি ": " 80 ",
    r"\(আশি |\(এইটি ": "(80 ",

    r"জিরো": "0",




    r"০": "0",
    r"১": "1",
    r"২": "2",
    r"৩": "3",
    r"৪": "4",
    r"৫": "5",
    r"৬": "6",
    r"৭": "7",
    r"৮": "8",
    r"৯": "9",

    r" এ ": " a ",                 # Replace "এ" with "a"
    r" বি ": " b ",                # Replace "বি" with "b"
    r" সি ": " c ",                # Replace "সি" with "c"
    r" ডি ": " d ",                # Replace "ডি" with "d"
    r" ই ": " e ",                 # Replace "ই" with "e"
    r" এফ ": " f ",                # Replace "এফ" with "f"
    r" জি ": " g ",                # Replace "জি" with "g"
    r" এইচ ": " h ",               # Replace "এইচ" with "h"
    r" আই ": " i ",                # Replace "আই" with "i"
    r" জে ": " j ",                # Replace "জে" with "j"
    r" কে ": " k ",                # Replace "কে" with "k"
    r" এল ": " l ",                # Replace "এল" with "l"
    r" এম ": " m ",                # Replace "এম" with "m"
    r" এন ": " n ",                # Replace "এন" with "n"
   # r"ও": "o",                 # Replace "ও" with "o"
    r" পি ": " p ",                # Replace "পি" with "p"
    r" কিউ ": " q ",               # Replace "কিউ" with "q"
    #r"আর": "r",                # Replace "আর" with "r"
    r" এস ": " s ",                # Replace "এস" with "s"
    r" টি ": " t ",                # Replace "টি" with "t"
    r" ইউ ": " u ",                # Replace "ইউ" with "u"
    r" ভি ": " v ",                # Replace "ভি" with "v"
    r" (ডব্লিউ।ডাবলিউ) ": " w ",            # Replace "ডব্লিউ" with "w"
    r" এক্স ": " x ",              # Replace "এক্স" with "x"
    r" ওয়াই | ওয়াই ": "y",             # Replace "ওয়াই" with "y"
    r" জেড ": " z ",               # Replace "জেড" with "z"


    #r"^এ ": "a ",                 # Replace "এ" with "a"
    r"^বি ": "b ",                # Replace "বি" with "b"
    r"^সি ": "c ",                # Replace "সি" with "c"
    r"^ডি ": "d ",                # Replace "ডি" with "d"
    r"^ই ": "e ",                 # Replace "ই" with "e"
    r"^এফ ": "f ",                # Replace "এফ" with "f"
    r"^জি ": "g ",                # Replace "জি" with "g"
    r"^এইচ ": "h ",               # Replace "এইচ" with "h"
    r"^আই ": "i ",                # Replace "আই" with "i"
    r"^জে ": "j ",                # Replace "জে" with "j"
    r"^কে ": "k ",                # Replace "কে" with "k"
    r"^এল ": "l ",                # Replace "এল" with "l"
    r"^এম ": "m ",                # Replace "এম" with "m"
    r"^এন ": "n ",                # Replace "এন" with "n"
   # r"ও": "o",                 # Replace "ও" with "o"
    r"^পি ": "p ",                # Replace "পি" with "p"
    r"^কিউ ": "q ",               # Replace "কিউ" with "q"
    #r"আর": "r",                # Replace "আর" with "r"
    r"^এস ": "s ",                # Replace "এস" with "s"
    r"^টি ": "t ",                # Replace "টি" with "t"
    r"^ইউ ": "u ",                # Replace "ইউ" with "u"
    r"^ভি ": "v ",                # Replace "ভি" with "v"
    r"^(ডব্লিউ।ডাবলিউ) ": "w ",            # Replace "ডব্লিউ" with "w"
    r"^এক্স ": "x ",              # Replace "এক্স" with "x"
    r"^ওয়াই |^ওয়াই ": "y ",              # Replace "ওয়াই" with "y"
    r"^জেড ": "z ",               # Replace "জেড" with "z"

    r" এ,": " a,",                 # Replace "এ" with "a"
    r" বি,": " b,",                # Replace "বি" with "b"
    r" সি,": " c,",                # Replace "সি" with "c"
    r" ডি,": " d,",                # Replace "ডি" with "d"
    r" ই,": " e,",                 # Replace "ই" with "e"
    r" এফ,": " f,",                # Replace "ফি" with "f"
    r" জি,": " g,",                # Replace "গি" with "g"
    r" এইচ,": " h,",               # Replace "এইচ" with "h"
    r" আই,": " i,",                # Replace "আই" with "i"
    r" জে,": " j,",                # Replace "জে" with "j"
    r" কে,": " k,",                # Replace "কে" with "k"
    r" এল,": " l,",                # Replace "এল" with "l"
    r" এম,": " m,",                # Replace "এম" with "m"
    r" এন,": " n,",                # Replace "এন" with "n"
    #r" ও,": " o,",                 # Replace "ও" with "o"
    r" পি,": " p,",                # Replace "পি" with "p"
    r" কিউ,": " q,",               # Replace "কিউ" with "q"
    #r" আর,": " r,",                # Replace "আর" with "r"
    r" এস,": " s,",                # Replace "এস" with "s"
    r" টি,": " t,",                # Replace "টি" with "t"
    r" ইউ,": " u,",                # Replace "ইউ" with "u"
    r" ভি,": " v,",                # Replace "ভি" with "v"
    r" (ডব্লিউ।ডাবলিউ),": " w,",            # Replace "ডব্লিউ" with "w"
    r" এক্স,": " x,",              # Replace "এক্স" with "x"
    r" ওয়াই,| ওয়াই,": " y,",              # Replace "ওয়াই" with "y"
    r" জেড,": " z,",               # Replace "জেড" with "z"

    r" এ।": " a।",                 # Replace "এ" with "a"
    r" বি।": " b।",                # Replace "বি" with "b"
    r" সি।": " c।",                # Replace "সি" with "c"
    r" ডি।": " d।",
    r" ই।": " e।",
    r" এফ।": " f।",
    r" জি।": " g।",
    r" এইচ।": " h।",
    r" আই।": " i।",
    r" জে।": " j।",
    r" কে।": " k।",
    r" এল।": " l।",
    r" এম।": " m।",
    r" এন।": " n।",
    r" ও।": " o।",
    r" পি।": " p।",
    r" কিউ।": " q।",
    r" আর।": " r।",
    r" এস।": " s।",
    r" টি।": " t।",
    r" ইউ।": " u।",
    r" ভি।": " v।",
    r" (ডব্লিউ।ডাবলিউ)।": " w।",
    r" এক্স।": " x।",
    r" ওয়াই।| ওয়াই।": " y।",
    r" জেড।": " z।",

    r" এ\)": " a)",                 # Replace "এ" with "a"
    r" বি\)": " b)",                # Replace "বি" with "b"
    r" সি\)": " c)",                # Replace "সি" with "c"
    r" ডি\)": " d)",
    r" ই\)": " e)",
    r" এফ\)": " f)",
    r" জি\)": " g)",
    r" এইচ\)": " h)",
    r" আই\)": " i)",
    r" জে\)": " j)",
    r" কে\)": " k)",
    r" এল\)": " l)",
    r" এম\)": " m)",
    r" এন\)": " n)",
    r" ও\)": " o)",
    r" পি\)": " p)",
    r" কিউ\)": " q)",
    r" আর\)": " r)",
    r" এস\)": " s)",
    r" টি\)": " t)",
    r" ইউ\)": " u)",
    r" ভি\)": " v)",
    r" (ডব্লিউ।ডাবলিউ)\)": " w)",
    r" এক্স\)": " x)",
    r" ওয়াই\)| ওয়াই\)": " y)",
    r" জেড\)": " z)",


    r"\(এ ": "\(a ",
    r"\(বি ": "\(b ",
r"\(সি ": "\(c ",
r"\(ডি ": "\(d ",
r"\(ই ": "\(e ",
r"\(এফ ": "\(f ",
r"\(জি ": "\(g ",
r"\(এইচ ": "\(h ",
r"\(আই ": "\(i ",
r"\(জে ": "\(j ",
r"\(কে ": "\(k ",
r"\(এল ": "\(l ",
r"\(এম ": "\(m ",
r"\(এন ": "\(n ",
r"\(ও ": "\(o ",
r"\(পি ": "\(p ",
r"\(কিউ ": "\(q ",
r"\(আর ": "\(r ",
r"\(এস ": "\(s ",
r"\(টি ": "\(t ",
r"\(ইউ ": "\(u ",
r"\(ভি ": "\(v ",
r"\((ডব্লিউ।ডাবলিউ) ": "\(w ",
r"\(এক্স ": "\(x ",
r"\(ওয়াই |\(ওয়াই ": "\(y ",
r"\(জেড ": "\(z ",

    r"ডিএসডি": "dsd",
    r"বিসিডি": "bcd",
    r"এসিডি": "acd",
    r"এবিসি": "abc",
    r"এসটিপি": "STP",
    r"এইচএসসি": "HSC",
    r"এসএসসি": "SSC",
    r"পিপিএম": "ppm",
    r"পিপিবি": "ppb",
    r"এসপি": "sp",
    r"এবি": "ab",
    r"এডি": "ad",
    r"এসসি": "sc",
    r"এসএ": "sa",
    r"এসকিউ": "sq",
    r"এসআর": "sr",
    r"এসটি": "st",
    r"এসএম": "sm",
    r"এসএন": "sn",
    r"এসও": "so",
    r"এসডি": "sd",
    r"এসই": "se",
    r"এসি ": "ac ",
    r"বিসি": "bc",
    r"পিকিউ": "pq",
    r"পিআর": "pr",
    r"পিএস": "ps",
    r"পিএম": "pm",
    r"পিএইচ": "pH",
    r"পিএ": "pa",

    r"পিবি": "pb",
    r"পিজেড": "pz",
    r"এক্সওয়াই|এক্সওয়াই": "xy",
    r"এক্সও": "xo",
    r"এক্সজেড": "xz",
    r"ওয়াইজেড|ওয়াইজেড": "yz",
    r"ওএ": "oa",
    r"ওবি": "ob",
    r"ওসি": "oc",
    r"ওডি": "od",
    r"ওপি": "op",
    r"ওকিউ": "oq",
    r"ওআর": "or",
    r"ওএস": "os",
    r"ইউভি": "uv",
    r"আইআর": "IR",
    r"ইএ": "ea",
    r"ইবি": "eb",
    r"কেপি": "kp",
    r"কেসি": "kc",
    r"এমসিকিউ": "MCQ",
    #r"সিকিউ": "CQ",
    r"ডিএন": "dn",
    r"ডিএস": "ds",
    r"ডিটি": "dt",
    r"ডিএক্স": "dx",
    r"ডিওয়াই|ডিওয়াই": "dy",
    r"ডিজেড": "dz",
    #r"সিপি": "Cp",
    #r"সিভি": "Cv",
    r"সিডি": "cd",
    r"সিসি": "cc",
    r"ডিডি": "dd",




    r"কসেক|কোসেক": "cosec",  # Replace "কসেক" or "কোসেক" with "cosec"
    r" লগ ": " log ",              # Replace "লগ" with "log"
    r"সাইন ": "sin ",            # Replace "সাইন" with "sin"
    r" কস । কজ ": " cos ",          # Replace "কজ" or "কস" with "cos"
    r"সেক ": "sec ",            # Replace "্সেক" with "sec"

    r"কট ": "cot ",              # Replace "কট" with "cot"
    r"ইনটু ": "X ",             # Replace "ইনটু" with "X"


    r"মিউ(?=\W|$)": "μ",               # "মিউ" → "μ"
    r"ডেলটা(?=\W|$)": "ẟ",             # "ডেলটা" → "ẟ"
    r"ল্যামডা(?=\W|$)": "λ",           # "ল্যামডা" → "λ"
    r"থিটা|থেটা(?=\W|$)": "θ",         # "থিটা" or "থেটা" → "θ"

    #r"পাই(?=\W|$)": "π",
    r"সিগমা(?=\W|$)": "σ",      # Sigma
   # r" রো(?=\W|$)": " ρ",         # Rho
    r"ওমেগা(?=\W|$)": "ω",      # Omega
    r" ফাই(?=\W|$)": " φ",        # Phi
    r"আলফা(?=\W|$)": "α",        # আলফা → α
    r"বিটা|বেটা(?=\W|$)": "β",    # বিটা/বেটা → β
    r"গামা(?=\W|$)": "γ",         # গামা → γ


    r"ন্যানোমিটার(?=\W|$)": "nm",
    r"মিলিমিটার(?=\W|$)": "mm",
    r"সেন্টিমিটার(?=\W|$)": "cm",
    r" মিটার(?=\W|$)": " m",
    r"কিলোমিটার(?=\W|$)": "km",

    r"সেকেন্ড(?=\W|$)": "s",
    r"মিলিসেকেন্ড(?=\W|$)": "ms",
    r"ন্যানোসেকেন্ড(?=\W|$)": "ns",
    r"মাইক্রোসেকেন্ড(?=\W|$)": "µs",

    r"কেজি(?=\W|$)": "kg",             # "কেজি" → "kg"
    r" গ্রাম(?=\W|$)": " g",
    r"মিলিগ্রাম(?=\W|$)": "mg",

     r"মাইনাস(?=\W|$)": "-",
     r"প্লাস(?=\W|$)": "+",
     r"কমা": ",",

    r"কার্বন ডাই অক্সাইড(?=\W|$)| কার্বন ডাইঅক্সাইড(?=\W|$)": "CO₂",
    r"অক্সিজেন গ্যাস(?=\W|$)": "O₂",
    r"হাইড্রোজেন গ্যাস(?=\W|$)": "H₂",
    r"নাইট্রোজেন গ্যাস(?=\W|$)": "N₂",
    r"ওজোন(?=\W|$)": "O₃",
    r"অ্যামোনিয়া (?=\W|$)|অ্যামোনিয়া (?=\W|$)": "NH₃ ",
    r"মিথেন(?=\W|$)": "CH₄",
    r"ইথেন(?=\W|$)": "C₂H₆",
    r"ইথিন(?=\W|$)|(অ্যাসিটিলিন|অ্যাসিটিলিন)(?=\W|$)": "C₂H₂",
    r"ইথানল(?=\W|$)": "C₂H₅OH",
    r"গ্লুকোজ(?=\W|$)": "C₆H₁₂O₆",
    r"সালফার ডাই অক্সাইড(?=\W|$)": "SO₂",
    r"সালফার ট্রাই অক্সাইড(?=\W|$)": "SO₃",
    r"হাইড্রোক্লোরিক (অ্যাসিড|অ্যাসিড)(?=\W|$)|হাইড্রোক্লোরিক এসিড(?=\W|$)|এইচসিএল(?=\W|$)": "HCl",
    r"নাইট্রিক (অ্যাসিড|অ্যাসিড)(?=\W|$)|নাইট্রিক এসিড(?=\W|$)": "HNO₃",
    r"সালফিউরিক (অ্যাসিড|অ্যাসিড)(?=\W|$)|সালফিউরিক এসিড(?=\W|$)": "H₂SO₄",
    r"(ক্যালসিয়াম|ক্যালসিয়াম) কার্বনেট(?=\W|$)": "CaCO₃",
    r"(সোডিয়াম|সোডিয়াম) ক্লোরাইড(?=\W|$)": "NaCl",
    r"(সোডিয়াম|সোডিয়াম) বাইকার্বোনেট(?=\W|$)": "NaHCO₃",
    r"(সোডিয়াম|সোডিয়াম) কার্বোনেট(?=\W|$)": "Na₂CO₃",
    r"(পটাসিয়াম|পটাসিয়াম) পারম্যাঙ্গানেট(?=\W|$)": "KMnO₄",
    r"হাইড্রোজেন পার অক্সাইড(?=\W|$)": "H₂O₂",

    r"হাইড্রোজেন(?=\W|$)": "H",
    r"(হিলিয়াম|হিলিয়াম)(?=\W|$)": "He",
    r"(লিথিয়াম|লিথিয়াম)(?=\W|$)": "Li",
    r"(বেরিলিয়াম|বেরিলিয়াম)(?=\W|$)": "Be",
    r"বোরন(?=\W|$)": "B",
    r"কার্বন (?=\W|$)": "C ",
    r"নাইট্রোজেন(?=\W|$)": "N",
    r"অক্সিজেন(?=\W|$)": "O",
    r"ফ্লোরিন(?=\W|$)": "F",
    r"(নিয়ন|নিয়ন)(?=\W|$)": "Ne",
    r"(সোডিয়াম|সোডিয়াম)(?=\W|$)": "Na",
    r"(ম্যাগনেসিয়াম|ম্যাগনেসিয়াম)(?=\W|$)": "Mg",
    r"(অ্যালুমিনিয়াম|অ্যালুমিনিয়াম)(?=\W|$)": "Al",
    r"সিলিকন(?=\W|$)": "Si",
    r"ফসফরাস(?=\W|$)": "P",
    r"সালফার(?=\W|$)": "S",
    r"ক্লোরিন(?=\W|$)": "Cl",
    r"আর্গন(?=\W|$)": "Ar",
    r"(পটাসিয়াম|পটাসিয়াম)(?=\W|$)": "K",
    r"(ক্যালসিয়াম|ক্যালসিয়াম)(?=\W|$)": "Ca",
    r"লোহা(?=\W|$)": "Fe",
    r"কপার|তামা(?=\W|$)": "Cu",
    r"জিঙ্ক(?=\W|$)": "Zn",
    r"সিলভার|রূপা(?=\W|$)": "Ag",
    r"গোল্ড|সোনা(?=\W|$)": "Au",
    r"মার্কারি|পারদ(?=\W|$)": "Hg",
    r"লেড|সিসা(?=\W|$)": "Pb",
    r" টিন(?=\W|$)": " Sn",
    r"নিকেল(?=\W|$)": "Ni",
    r"(ক্রোমিয়াম|ক্রোমিয়াম)(?=\W|$)": "Cr",
    r"ম্যাঙ্গানিজ(?=\W|$)": "Mn",
    r"কোবাল্ট(?=\W|$)": "Co",
    r"প্লাটিনাম(?=\W|$)": "Pt",
    r"(ইউরেনিয়াম|ইউরেনিয়াম)(?=\W|$)": "U",
    r"(আয়োডিন|আয়োডিন)(?=\W|$)": "I",
    r"ব্রোমিন(?=\W|$)": "Br",

    r"ফিজিক্স(?=\W|$)": "Physics",
    r"(রসায়ন|রসায়ন)(?=\W|$)|কেমিস্ট্রি(?=\W|$)": "Chemistry",
    r"জীববিজ্ঞান(?=\W|$)|(বায়োলজি|বায়োলজি)(?=\W|$)": "Biology",
    r"উচ্চতর গণিত(?=\W|$)|(হায়ার|হায়ার) ম্যাথ(?=\W|$)|(হায়ার|হায়ার) ম্যাথমেটিক্স(?=\W|$)": "Higher Mathematics",
    r"ম্যাথ(?=\W|$)": "Math",
    r"ইংরেজি(?=\W|$)": "English",
    r"বাংলা(?=\W|$)": "Bangla",
    r"আইসিটি(?=\W|$)|তথ্য ও যোগাযোগ প্রযুক্তি(?=\W|$)": "ICT",
    r"ভূগোল(?=\W|$)|জিওগ্রাফি(?=\W|$)": "Geography",
    r"সাইন্স(?=\W|$)": "Science",
    r"কম্পিউটার(?=\W|$)": "Computer",
    r"এই অংশে ": ""
}


                for pattern, replacement in regex_patterns.items():
                  final_text = re.sub(pattern, replacement, final_text)
                  regex_pat={
    r"([a-zA-Z₀₁₂₃₄₅₆₇₈₉+-])ে": r"\1এ",
    r"([a-z0-9λθπσρωφαβγ]) (স্কয়ার|স্কয়ার|স্কোয়ার|স্কোয়ার)": r"\1²",
    r"([a-z0-9λθπσρωφαβγ]) কিউব": r"\1³",
    r"(রুট ওভার।রুট) ([a-z0-9λθπσρωφαβγ])": r"√\2",
    r"([a-zλθπσρωφαβγ]) নট": r"\1₀",
    r"টান ([a-zλθπσρωφαβγ])": r"tan \1",
    r"(কস|কজ) ([a-zλθπσρωφαβγ])": r"cos \2"



}

                for pattern, replacement in regex_pat.items():
                  final_text = re.sub(pattern, replacement, final_text)

            # Save final text
            with open(final_text_path, 'w', encoding='utf-8') as f:
              f.write(final_text)


            # Final update
            update_status(100, f'Processing completed successfully! ({total_iterations} summarization iterations)')


        # Return final results
            results = {
            'status': 'completed',
            'progress': 100,
            'message': f'Processing completed successfully! ({total_iterations} summarization iterations)',
            'files': {
                'audio': audio_path,
                'transcription': transcription_path,
                'summary_excel': summary_path,
                'final_text': final_text_path
            },
            'preview': final_text[:5000] + "..." if len(final_text) > 5000 else final_text,
            'iterations': total_iterations
        }

            return results

        except Exception as e:

            error_msg = f'Processing failed: {str(e)}'
            if job_id and job_id in processing_status:
              processing_status[job_id].update({
                  'status': 'error',
                 'message': error_msg,
                  'error': traceback.format_exc()
              })
            return {
            'status': 'error',
            'message': error_msg,
            'error': traceback.format_exc()
         }


    def recursive_summarization_with_concatenation(self, df, max_chunks=50):
        """
        Recursively summarize by concatenating pairs of rows until we have <= max_chunks
        """
        current_df = df.copy()
        iteration = 1

        while len(current_df) > max_chunks:
            print(f"\n🔄 Iteration {iteration}: Processing {len(current_df)} rows...")

            # Generate summaries for current rows
            print("📄 Generating summaries...")
            current_df['Summary'] = current_df['Text'].apply(
                lambda x: self.generate_summary(str(x))
            )

            # Calculate token counts if tokenizer is available
            if hasattr(self, 'tokenizer') and self.tokenizer:
                print("🔢 Calculating token counts...")
                current_df['Token(Text)'] = current_df['Text'].apply(
                    lambda x: self.count_tokens(x)
                )
                current_df['Token(Summary)'] = current_df['Summary'].apply(
                    lambda x: self.count_tokens(x)
                )

                # Print token statistics
                avg_text_tokens = current_df['Token(Text)'].mean()
                avg_summary_tokens = current_df['Token(Summary)'].mean()
                print(f"📊 Token Statistics - Iteration {iteration}:")
                print(f"   Average Text tokens: {avg_text_tokens:.1f}")
                print(f"   Average Summary tokens: {avg_summary_tokens:.1f}")

            # Prepare dataframe for saving with required columns
            save_columns = ['Text', 'Summary', 'Start_Time', 'End_Time']
            if hasattr(self, 'tokenizer') and self.tokenizer:
                save_columns.extend(['Token(Text)', 'Token(Summary)'])

            save_df = current_df[save_columns].copy()

            # Save Excel file for this iteration
            iteration_excel_path = os.path.join(
                os.path.dirname(self.temp_dir) if hasattr(self, 'temp_dir') else '/tmp',
                f"iteration_{iteration}_results.xlsx"
            )
            save_df.to_excel(iteration_excel_path, index=False)
            print(f"💾 Iteration {iteration} results saved: {iteration_excel_path}")

            # Concatenate pairs of rows
            print("🔗 Concatenating pairs of rows...")
            current_df = self.concatenate_pairs(current_df)

            print(f"🔢 After concatenation: {len(current_df)} rows")

            # Debug: Print sample data
            print(f"📝 Sample rows after concatenation:")
            for i, (idx, row) in enumerate(current_df.head(3).iterrows()):
                print(f"   Row {i+1} (ID: {row.get('Chunk_ID', 'N/A')}):")
                print(f"     Start: {row.get('Start_Time', 'N/A')}, End: {row.get('End_Time', 'N/A')}")
                print(f"     Text: {str(row.get('Text', ''))[:100]}...")

            iteration += 1

            # Safety check to prevent infinite loops
            if iteration > 10:
                print("⚠️ Maximum iterations reached. Breaking loop.")
                break

        # Final summarization
        print(f"\n✅ Final iteration: Processing {len(current_df)} rows...")
        current_df['Final_Summary'] = current_df['Text'].apply(
            lambda x: self.generate_summary(str(x))
        )

        return current_df, iteration
# Initialize the processor
try:
    print("🔄 Initializing model...")
    processor = VideoProcessor()
    print("✅ VideoProcessor initialized successfully!")
except Exception as e:
    print(f"❌ Model initialization failed: {e}")
    print("Please check your model path or use a different model")
    processor = None



# Flask Application
app = Flask(__name__)
CORS(app)

# Global variables
processor = None
processing_status = {}

# Simple HTML template for Colab
HTML_TEMPLATE = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Video Processing & Summarization</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; background: #f5f5f5; }
        .container { max-width: 800px; margin: 0 auto; background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
        .upload-area { border: 2px dashed #4CAF50; padding: 30px; text-align: center; margin: 20px 0; border-radius: 10px; background: #f9fff9; }
        .upload-area:hover { background: #f0fff0; }
        .progress-bar { width: 100%; height: 25px; background: #f0f0f0; border-radius: 12px; overflow: hidden; margin: 10px 0; }
        .progress-fill { height: 100%; background: linear-gradient(90deg, #4CAF50, #45a049); width: 0%; transition: width 0.3s; }
        button { background: #007bff; color: white; border: none; padding: 12px 24px; border-radius: 6px; cursor: pointer; font-size: 16px; margin: 5px; }
        button:hover { background: #0056b3; }
        button:disabled { background: #ccc; cursor: not-allowed; }
        .download-btn { background: #28a745; }
        .download-btn:hover { background: #218838; }
        .error { color: #dc3545; padding: 15px; background: #f8d7da; border: 1px solid #f5c6cb; border-radius: 5px; margin: 10px 0; }
        .success { color: #155724; padding: 15px; background: #d4edda; border: 1px solid #c3e6cb; border-radius: 5px; margin: 10px 0; }
        .preview { max-height: 300px; overflow-y: auto; border: 1px solid #ddd; padding: 15px; background: #f8f9fa; border-radius: 5px; font-family: monospace; }
        .status-badge { display: inline-block; padding: 5px 10px; border-radius: 15px; font-size: 12px; font-weight: bold; }
        .status-processing { background: #fff3cd; color: #856404; }
        .status-completed { background: #d4edda; color: #155724; }
        .status-error { background: #f8d7da; color: #721c24; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎬 Video Processing & Summarization</h1>
        <p>Upload a Bengali video file to get automated transcription and summarization</p>

        <div class="upload-area" id="uploadArea" onclick="document.getElementById('videoInput').click()">
            <input type="file" id="videoInput" accept="video/*" style="display: none;">
            <p>📁 Click to select a video file or drag and drop</p>
            <button type="button">Choose Video File</button>
        </div>

        <div id="fileInfo" style="display: none;">
            <h3>📹 Selected File</h3>
            <p><strong>Name:</strong> <span id="fileName"></span></p>
            <p><strong>Size:</strong> <span id="fileSize"></span></p>
            <button id="processBtn" onclick="processVideo()">🚀 Start Processing</button>
        </div>

        <div id="progressSection" style="display: none;">
            <h3>⚙️ Processing Status</h3>
            <div class="progress-bar">
                <div class="progress-fill" id="progressFill"></div>
            </div>
            <p id="progressText">0% - Initializing...</p>
        </div>

        <div id="resultsSection" style="display: none;">
            <h3>📊 Processing Results</h3>
            <div id="downloadButtons" style="text-align: center; margin: 20px 0;">
                <button class="download-btn" onclick="downloadFile('audio')">🎵 Audio File</button>
                <button class="download-btn" onclick="downloadFile('transcription')">📋 Transcription</button>
                <button class="download-btn" onclick="downloadFile('summary_excel')">📊 Summary Excel</button>
                <button class="download-btn" onclick="downloadFile('final_text')">📝 Final Summary</button>
            </div>

            <h4>🔍 Preview:</h4>
            <div id="summaryPreview" class="preview"></div>
        </div>

        <div id="messages"></div>
    </div>

    <script>
        let selectedFile = null;
        let currentJobId = null;

        document.getElementById('videoInput').addEventListener('change', function(e) {
            const file = e.target.files[0];
            if (file) {
                selectedFile = file;
                document.getElementById('fileName').textContent = file.name;
                document.getElementById('fileSize').textContent = (file.size / 1024 / 1024).toFixed(2) + ' MB';
                document.getElementById('fileInfo').style.display = 'block';
            }
        });

        async function processVideo() {
            if (!selectedFile) {
                showError('Please select a video file first');
                return;
            }

            const formData = new FormData();
            formData.append('video', selectedFile);

            document.getElementById('progressSection').style.display = 'block';
            const processBtn = document.getElementById('processBtn');
            processBtn.disabled = true;
            processBtn.textContent = '⏳ Processing...';

            try {
                const response = await fetch('/api/process', {
                    method: 'POST',
                    body: formData
                });

                const result = await response.json();

                if (result.status === 'success') {
                    currentJobId = result.job_id;
                    pollProgress(result.job_id);
                } else {
                    throw new Error(result.message);
                }

            } catch (error) {
                showError('Processing failed: ' + error.message);
                processBtn.disabled = false;
                processBtn.textContent = '🚀 Start Processing';
            }
        }

        async function pollProgress(jobId) {
            const interval = setInterval(async () => {
                try {
                    const response = await fetch(`/api/status/${jobId}`);
                    const status = await response.json();

                    updateProgress(status.progress || 0, status.message || 'Processing...');

                    if (status.status === 'completed') {
                        clearInterval(interval);
                        displayResults(status);
                        showSuccess('🎉 Video processing completed successfully!');
                        document.getElementById('resultsSection').style.display = 'block';
                        document.getElementById('processBtn').disabled = false;
                        document.getElementById('processBtn').textContent = '🚀 Start Processing';
                    } else if (status.status === 'error') {
                        clearInterval(interval);
                        showError('❌ Processing failed: ' + status.message);
                        document.getElementById('processBtn').disabled = false;
                        document.getElementById('processBtn').textContent = '🚀 Start Processing';
                    }
                } catch (error) {
                    console.error('Error polling status:', error);
                }
            }, 3000);
        }

        function updateProgress(progress, message) {
            document.getElementById('progressFill').style.width = progress + '%';
            document.getElementById('progressText').textContent = progress + '% - ' + message;
        }

        function displayResults(status) {
            if (status.preview) {
                document.getElementById('summaryPreview').textContent = status.preview;
            }
        }

        async function downloadFile(type) {
            if (!currentJobId) {
                showError('No processing job found');
                return;
            }

            try {
                const response = await fetch(`/api/download/${currentJobId}/${type}`);

                if (response.ok) {
                    const blob = await response.blob();
                    const url = window.URL.createObjectURL(blob);
                    const a = document.createElement('a');
                    a.href = url;
                    a.download = getFileName(type);
                    a.click();
                    window.URL.revokeObjectURL(url);
                } else {
                    showError('Download failed');
                }
            } catch (error) {
                showError('Download error: ' + error.message);
            }
        }

        function getFileName(type) {
            const names = {
                'audio': 'extracted_audio.wav',
                'transcription': 'transcription_chunks.xlsx',
                'summary_excel': 'summaries_with_timestamps.xlsx',
                'final_text': 'final_summary.txt'
            };
            return names[type] || 'download.txt';
        }

        function showError(message) {
            const div = document.createElement('div');
            div.className = 'error';
            div.innerHTML = '❌ ' + message;
            document.getElementById('messages').appendChild(div);
            setTimeout(() => div.remove(), 8000);
        }

        function showSuccess(message) {
            const div = document.createElement('div');
            div.className = 'success';
            div.innerHTML = '✅ ' + message;
            document.getElementById('messages').appendChild(div);
            setTimeout(() => div.remove(), 8000);
        }
    </script>
</body>
</html>'''

@app.route('/')
def index():
    """Serve the main HTML page"""
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/process', methods=['POST'])
def process_video():
    """Start video processing"""
    try:
        if 'video' not in request.files:
            return jsonify({'status': 'error', 'message': 'No video file provided'})

        video_file = request.files['video']
        if video_file.filename == '':
            return jsonify({'status': 'error', 'message': 'No video file selected'})

        # Create unique job ID
        job_id = str(int(time.time() * 1000))

        # Create temporary directory for this job
        temp_dir = os.path.join('/tmp', f'video_processing_{job_id}')
        os.makedirs(temp_dir, exist_ok=True)

        # Save uploaded file
        video_path = os.path.join(temp_dir, video_file.filename)
        video_file.save(video_path)

        # Initialize processing status
        processing_status[job_id] = {
            'status': 'queued',
            'progress': 0,
            'message': 'Processing queued...',
            'temp_dir': temp_dir
        }

        # Start processing in background thread
        thread = threading.Thread(
            target=background_process,
            args=(job_id, video_path, temp_dir)
        )
        thread.daemon = True
        thread.start()

        return jsonify({
            'status': 'success',
            'job_id': job_id,
            'message': 'Processing started'
        })

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        })

def background_process(job_id, video_path, temp_dir):
    """Background processing function"""
    try:
        global processor
        if processor is None:
            processing_status[job_id].update({
                'status': 'error',
                'message': 'Model not initialized'
            })
            return

        # Update status to processing
        processing_status[job_id].update({
            'status': 'processing',
            'progress': 5,
            'message': 'Starting video processing...'
        })

        # Process the video
        result = processor.process_complete_pipeline(video_path, temp_dir, job_id=job_id)

        # Update final status
        processing_status[job_id].update(result)

    except Exception as e:
        processing_status[job_id].update({
            'status': 'error',
            'message': str(e),
            'error': traceback.format_exc()
        })

@app.route('/api/status/<job_id>')
def get_status(job_id):
    """Get processing status"""
    if job_id in processing_status:
        return jsonify(processing_status[job_id])
    else:
        return jsonify({
            'status': 'error',
            'message': 'Job not found'
        })

@app.route('/api/download/<job_id>/<file_type>')
def download_file(job_id, file_type):
    """Download processed files"""
    if job_id not in processing_status:
        return jsonify({'error': 'Job not found'}), 404

    status = processing_status[job_id]
    if status['status'] != 'completed':
        return jsonify({'error': 'Processing not completed'}), 400

    if 'files' not in status or file_type not in status['files']:
        return jsonify({'error': 'File not found'}), 404

    file_path = status['files'][file_type]
    if not os.path.exists(file_path):
        return jsonify({'error': f'File does not exist: {file_path}'}), 404

    # Define MIME types and download names for different file types
    file_configs = {
        'audio': {
            'mimetype': 'audio/wav',
            'download_name': 'extracted_audio.wav'
        },
        'transcription': {
            'mimetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'download_name': 'transcription_chunks.xlsx'
        },
        'summary_excel': {
            'mimetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            'download_name': 'summaries_with_timestamps.xlsx'
        },
        'final_text': {
            'mimetype': 'text/plain; charset=utf-8',
            'download_name': 'final_summary.txt'
        }
    }

    config = file_configs.get(file_type, {
        'mimetype': 'application/octet-stream',
        'download_name': f'download_{file_type}'
    })

    try:
        return send_file(
            file_path,
            as_attachment=True,
            download_name=config['download_name'],
            mimetype=config['mimetype']
        )
    except Exception as e:
        print(f"Error sending file {file_path}: {e}")
        return jsonify({'error': f'Failed to send file: {str(e)}'}), 500

# ===== CELL 4: Initialize and Run =====
# Initialize the processor
try:
    print("🔄 Initializing model...")
    processor = VideoProcessor()  # Will use default model
    print("✅ Model initialized successfully!")
except Exception as e:
    print(f"❌ Model initialization failed: {e}")
    print("Please check your model path or use a different model")

# Start the Flask app with ngrok
if __name__ == '__main__':
    # Create ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f"🌐 Public URL: {public_url}")
    print(f"🔗 Access your app at: {public_url}")

    # Run Flask app
    app.run(host='0.0.0.0', port=5000, debug=False)





















































  r"\(এ ": "\(a ",

  r"\(বি ": "\(b ",

  r"\(সি ": "\(c ",

  r"\(ডি ": "\(d ",

  r"\(ই ": "\(e ",

  r"\(এফ ": "\(f ",

  r"\(জি ": "\(g ",

  r"\(এইচ ": "\(h ",

  r"\(আই ": "\(i ",

  r"\(জে ": "\(j ",

  r"\(কে ": "\(k ",

  r"\(এল ": "\(l ",

  r"\(এম ": "\(m ",

  r"\(এন ": "\(n ",

  r"\(ও ": "\(o ",

  r"\(পি ": "\(p ",

  r"\(কিউ ": "\(q ",

  r"\(আর ": "\(r ",

  r"\(এস ": "\(s ",

  r"\(টি ": "\(t ",

  r"\(ইউ ": "\(u ",

  r"\(ভি ": "\(v ",

  r"\((ডব্লিউ।ডাবলিউ) ": "\(w ",

  r"\(এক্স ": "\(x ",

  r"\(ওয়াই |\(ওয়াই ": "\(y ",

  r"\(জেড ": "\(z ",



  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-uw0w2kg8
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-uw0w2kg8
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.l

Device set to use cuda:0


✅ BanglaPunctuation loaded
✅ VideoProcessor initialized successfully!
🔄 Initializing model...


Device set to use cuda:0


...

INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:34:17] "GET /api/status/1756468506036 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:35:17] "GET /api/status/1756468506036 HTTP/1.1" 200 -


DataFrame shape: (28, 15)
Number of rows: 28
File saved successfully to: /content/drive/MyDrive/tfidf/new.txt
✅ File copied to: /content/drive/MyDrive/tfidf/Concatenated_Summary/new.txt
Initializing Enhanced Bangla Combined Keyword Extractor...
Error initializing BNLP components: name 'BasicTokenizer' is not defined
Using fallback tokenization (simple split)
Successfully loaded 970 stopwords (no tokenization - tokenizer not available)

Loading documents from: /content/drive/MyDrive/tfidf/Concatenated_Summary
Found 213 .txt files.
Building corpus vocabulary...
Built corpus vocabulary: 13814 prominent terms
Processing documents with combined unigram-bigram extraction...
Added document: sum2.txt with 4902 unigrams, 2319 bigrams
Added document: sum22.txt with 1861 unigrams, 1002 bigrams
Added document: sum45.txt with 1509 unigrams, 880 bigrams
Added document: sum51.txt with 2662 unigrams, 1650 bigrams
Added document: sum3.txt with 3819 unigrams, 2068 bigrams
Added document: sum9.txt with 3

INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:36:17] "GET /api/status/1756468506036 HTTP/1.1" 200 -


Document length: 2006 tokens, Dynamic keywords: 40
Document length: 4042 tokens, Dynamic keywords: 50
Document length: 1580 tokens, Dynamic keywords: 31
Document length: 127 tokens, Dynamic keywords: 5
Document length: 1590 tokens, Dynamic keywords: 31
Document length: 1349 tokens, Dynamic keywords: 26
Document length: 163 tokens, Dynamic keywords: 5
Document length: 2367 tokens, Dynamic keywords: 47
Document length: 3105 tokens, Dynamic keywords: 50
Document length: 303 tokens, Dynamic keywords: 6
Document length: 1968 tokens, Dynamic keywords: 39
Document length: 1730 tokens, Dynamic keywords: 34
Document length: 180 tokens, Dynamic keywords: 5
Document length: 116 tokens, Dynamic keywords: 5
Document length: 3304 tokens, Dynamic keywords: 50
Document length: 1113 tokens, Dynamic keywords: 22
Document length: 1821 tokens, Dynamic keywords: 36
Document length: 2243 tokens, Dynamic keywords: 44
Document length: 3029 tokens, Dynamic keywords: 50
Document length: 1705 tokens, Dynamic key

INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:37:16] "GET /api/status/1756468506036 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:37:17] "GET /api/status/1756468506036 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:37:23] "GET /api/download/1756468506036/final_text HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:37:52] "GET /api/download/1756468506036/summary_excel HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:39:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Aug/2025 12:40:03] "GET / HTTP/1.1" 200 -
